[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/PPatty666/Germany_Flood_Study/blob/main/Visuals/visual_scripts/Cleaning_up_Germany_Geimende_plots_maps.ipynb
)

In [ ]:
%%capture

!pip install fiona
!pip install folium mapclassify
!pip install ipyleaflet
!pip install -U kaleido
!pip install contextily
!pip install git+https://github.com/pmdscully/geo_northarrow.git
!pip install matplotlib_scalebar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.ticker as mticker
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from matplotlib.colors import to_hex
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from moviepy.editor import ImageSequenceClip
from PIL import Image
from IPython.display import Video

from ipyleaflet import Map, GeoJSON, Popup
from ipywidgets import HTML
import json
import folium
from folium import IFrame

import re
import math
from numpy.random import default_rng
import scipy
from scipy.stats import gamma
import statsmodels.api as sm
# import arviz as az
# import pymc as pm
# import xarray as xr

import geopandas as gpd
import fiona
from shapely.geometry import MultiLineString, LineString
from shapely.geometry import Polygon, MultiPolygon
import contextily as ctx
from geo_northarrow import add_north_arrow
from matplotlib_scalebar.scalebar import ScaleBar
import mapclassify
from mapclassify import NaturalBreaks
from mapclassify import Quantiles
from matplotlib.lines import Line2D


from shapely.ops import unary_union

import pickle
import os

import warnings
warnings.filterwarnings('ignore')

### Load shape file for mapping, preliminary check for spatial join and some dataset pre-processing: **PLEASE update the file path to read in the raw file**

In [ ]:
%%capture
!unzip /content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/vg250_01-01.gk3.shape.ebenen.zip -d admin-GIS

In [ ]:
#Extract excel files at Gemeinden level
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/Gemeinde_Stats_all_return_periods.zip"

### Please run this section every time you run out of ram and need to restart the session

In [ ]:
admin3_boundary = gpd.read_file('/content/admin-GIS/vg250_01-01.gk3.shape.ebenen/vg250_ebenen_0101/VG250_GEM.shp')

In [ ]:
#Read raw files
file_paths = [
    '/content/GermanyStats_2020_5.csv',
    '/content/GermanyStats_2020_10.csv',
    '/content/GermanyStats_2020_20.csv',
    '/content/GermanyStats_2020_50.csv',
    '/content/GermanyStats_2020_100.csv',
    '/content/GermanyStats_2020_200.csv',
    '/content/GermanyStats_2020_500.csv',
    '/content/GermanyStats_2020_1000.csv'
]

dataframes = []
for path in file_paths:
    df = pd.read_csv(path, encoding='ISO-8859-1')
    df = df.round(6)
    dataframes.append(df)

# Optionally unpack into variables
(Gemeinde_2020_5_raw, Gemeinde_2020_10_raw, Gemeinde_2020_20_raw, Gemeinde_2020_50_raw,
 Gemeinde_2020_100_raw, Gemeinde_2020_200_raw, Gemeinde_2020_500_raw, Gemeinde_2020_1000_raw) = dataframes

In [ ]:
Gemeinde_2020_5_raw.head()

In [ ]:
# Perform the join
Gemeinde_2020_5_raw_gdf = admin3_boundary.merge(Gemeinde_2020_5_raw, left_on='OBJID', right_on='GEM_ID', how='right')

In [ ]:
# Check table join
len(Gemeinde_2020_5_raw_gdf)

In [ ]:
# Check table join
Gemeinde_2020_5_raw_gdf[Gemeinde_2020_5_raw_gdf.geometry.isna()]

In [ ]:
# Check table join
len(Gemeinde_2020_5_raw_gdf.geometry.unique())

In [ ]:
# Create the list of new month_year labels (6-month intervals)
month_years = pd.date_range('2016-07-01', '2025-01-01', freq='6MS').strftime('%b-%y').tolist()
print(month_years)

In [ ]:
# Get current column names
cols = Gemeinde_2020_5_raw.columns.tolist()
cols

In [ ]:
# Update column names
new_cols = []
for col in cols:
    match = re.search(r'_(\d{2})$', col)
    if match:
        suffix_num = int(match.group(1))
        if 1 <= suffix_num <= len(month_years):
            new_suffix = month_years[suffix_num - 1]  # map _01 to Jul-16, etc.
            col = re.sub(r'_(\d{2})$', f'_{new_suffix}', col)
    new_cols.append(col)

In [ ]:
new_cols

In [ ]:
# Assign new column names back to the DataFrame
for df in [Gemeinde_2020_5_raw, Gemeinde_2020_10_raw, Gemeinde_2020_20_raw, Gemeinde_2020_50_raw,
           Gemeinde_2020_100_raw, Gemeinde_2020_200_raw, Gemeinde_2020_500_raw, Gemeinde_2020_1000_raw]:
    df.columns = new_cols

## Settlement area (Please run this section every time you run out of ram and need to restart the session)

In [ ]:
sa_columns = new_cols[:25]

years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_SA'] = globals()[f'Gemeinde_2020_{y}_raw'][sa_columns]

In [ ]:
Gemeinde_2020_5_SA.head()

In [ ]:
#Create long data format -- Exposure from different month-year will now be stored in a single column
years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_SA_long'] = globals()[f'Gemeinde_2020_{y}_SA'].melt(
        id_vars=new_cols[:7], var_name='month_year', value_name='Settlement_Area'
    )

In [ ]:
Gemeinde_2020_5_SA_long.head()

In [ ]:
for df in [Gemeinde_2020_5_SA_long, Gemeinde_2020_10_SA_long, Gemeinde_2020_20_SA_long, Gemeinde_2020_50_SA_long,\
           Gemeinde_2020_100_SA_long, Gemeinde_2020_200_SA_long, Gemeinde_2020_500_SA_long, Gemeinde_2020_1000_SA_long]:
    df['month_year'] = df['month_year'].str.replace(r'^SA_', '', regex=True)

In [ ]:
Gemeinde_2020_5_SA_long.head()

# For each flooding-type section, if the notebook runs out of RAM and the session must be restarted, please resume from the beginning of that specific flooding-type section. For example, if the session restarts while running **“Fluvial flooding undefended”**, begin again from the **data preparation** step of that section after reinstalling and importing the required libraries and re-running all necessary setup cells noted above. Thank you!

## Combined(max) flooding

### data preparation

In [ ]:
# Extract columns that stores the combined flooding depth
max_columns = new_cols[:7] + [col for col in new_cols if col.startswith('SA_maxRisk')]

years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_MAX'] = globals()[f'Gemeinde_2020_{y}_raw'][max_columns]


In [ ]:
years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_MAX_long'] = globals()[f'Gemeinde_2020_{y}_MAX'].melt(
        id_vars=new_cols[:7], var_name='month_year_depth', value_name='MAX_area'
    )

In [ ]:
Gemeinde_2020_5_MAX_long.head()

In [ ]:
for df in [Gemeinde_2020_5_MAX_long, Gemeinde_2020_10_MAX_long, Gemeinde_2020_20_MAX_long, Gemeinde_2020_50_MAX_long,
           Gemeinde_2020_100_MAX_long, Gemeinde_2020_200_MAX_long, Gemeinde_2020_500_MAX_long, Gemeinde_2020_1000_MAX_long]:

    # Extract depth category (e.g., '0', '015', '050', '150', '150p') and month-year (e.g., 'Jul-16')
    df[['depth_cat', 'month_year']] = df['month_year_depth'].str.extract(r'_([0-9]{1,4}p?)_([A-Za-z]{3}-\d{2})$')

In [ ]:
Gemeinde_2020_5_MAX_long.head()

In [ ]:
# Merge the flooding exposure with the total settlement area to calculate the percentage of the exposed settlement area
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    long_df = globals()[f'Gemeinde_2020_{y}_MAX_long']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][['GEM_ID', 'month_year', 'Settlement_Area']]

    merged_df = pd.merge(long_df, settle_df, on=['GEM_ID', 'month_year'], how='left')

    globals()[f'Gemeinde_2020_{y}_MAX_df'] = merged_df


In [ ]:
Gemeinde_2020_5_MAX_df.head()

In [ ]:
for df in [Gemeinde_2020_5_MAX_df, Gemeinde_2020_10_MAX_df, Gemeinde_2020_20_MAX_df, Gemeinde_2020_50_MAX_df,
           Gemeinde_2020_100_MAX_df, Gemeinde_2020_200_MAX_df, Gemeinde_2020_500_MAX_df, Gemeinde_2020_1000_MAX_df]:
    df['depth_cat_area_perc'] = (df.MAX_area / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_MAX_baseline = Gemeinde_2020_5_MAX_df[Gemeinde_2020_5_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_10_MAX_baseline = Gemeinde_2020_10_MAX_df[Gemeinde_2020_10_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_20_MAX_baseline = Gemeinde_2020_20_MAX_df[Gemeinde_2020_20_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_50_MAX_baseline = Gemeinde_2020_50_MAX_df[Gemeinde_2020_50_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_100_MAX_baseline = Gemeinde_2020_100_MAX_df[Gemeinde_2020_100_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_200_MAX_baseline = Gemeinde_2020_200_MAX_df[Gemeinde_2020_200_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_500_MAX_baseline = Gemeinde_2020_500_MAX_df[Gemeinde_2020_500_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]
Gemeinde_2020_1000_MAX_baseline = Gemeinde_2020_1000_MAX_df[Gemeinde_2020_1000_MAX_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'MAX_area']]

In [ ]:
for y in [5, 10, 20, 50, 100, 200, 500, 1000]:
    df_name = f'Gemeinde_2020_{y}_MAX_baseline'
    df = globals()[df_name]
    globals()[df_name] = df.rename(columns={'MAX_area': 'MAX_area_Jul16'})

In [ ]:
# Create baseline dataframes with the flooding exposure area in July 2016, as this is refered as the starting point to compare with
for df_name in ['Gemeinde_2020_5_MAX_baseline', 'Gemeinde_2020_10_MAX_baseline', 'Gemeinde_2020_20_MAX_baseline', 'Gemeinde_2020_50_MAX_baseline',
                'Gemeinde_2020_100_MAX_baseline', 'Gemeinde_2020_200_MAX_baseline', 'Gemeinde_2020_500_MAX_baseline', 'Gemeinde_2020_1000_MAX_baseline']:
    df = globals()[df_name]
    df = df.rename(columns={'MAX_area': 'MAX_area_Jul16'})  # Rename first
    df['MAX_area_Jul16_non_zero'] = df['MAX_area_Jul16'].replace(0, 1e-6)  # Replace zeros
    globals()[df_name] = df

In [ ]:
Gemeinde_2020_5_MAX_baseline.head()

In [ ]:
for return_period in [5, 10, 20, 50, 100, 200, 500, 1000]:
    orig_df = globals()[f'Gemeinde_2020_{return_period}_MAX_df']
    baseline_df = globals()[f'Gemeinde_2020_{return_period}_MAX_baseline']
    merged_df = orig_df.merge(baseline_df, on=['GEM_ID', 'depth_cat'], how='left')
    globals()[f'Gemeinde_2020_{return_period}_MAX_df'] = merged_df

In [ ]:
Gemeinde_2020_5_MAX_df.head()

In [ ]:
for df in [Gemeinde_2020_5_MAX_df, Gemeinde_2020_10_MAX_df, Gemeinde_2020_20_MAX_df, Gemeinde_2020_50_MAX_df,
           Gemeinde_2020_100_MAX_df, Gemeinde_2020_200_MAX_df, Gemeinde_2020_500_MAX_df, Gemeinde_2020_1000_MAX_df]:
    df['MAX_area_non_zero'] = df.MAX_area.replace(0, 1e-6)
    df['pct_growth'] = (df['MAX_area_non_zero'] - df['MAX_area_Jul16_non_zero']) / df['MAX_area_Jul16_non_zero'] * 100

### Create risk level - low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
def risk_level(value):
    if value == '0' or value == '015':
        return 'low_risk'
    else:
        return 'high_risk'

In [ ]:
for df in [Gemeinde_2020_5_MAX_df, Gemeinde_2020_10_MAX_df, Gemeinde_2020_20_MAX_df, Gemeinde_2020_50_MAX_df,
           Gemeinde_2020_100_MAX_df, Gemeinde_2020_200_MAX_df, Gemeinde_2020_500_MAX_df, Gemeinde_2020_1000_MAX_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Gemeinde_2020_5_MAX_df.head()

In [ ]:
Gemeinde_2020_5_jan25_MAX = Gemeinde_2020_5_MAX_df[Gemeinde_2020_5_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_10_jan25_MAX = Gemeinde_2020_10_MAX_df[Gemeinde_2020_10_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_20_jan25_MAX = Gemeinde_2020_20_MAX_df[Gemeinde_2020_20_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_50_jan25_MAX = Gemeinde_2020_50_MAX_df[Gemeinde_2020_50_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_100_jan25_MAX = Gemeinde_2020_100_MAX_df[Gemeinde_2020_100_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_200_jan25_MAX = Gemeinde_2020_200_MAX_df[Gemeinde_2020_200_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_500_jan25_MAX = Gemeinde_2020_500_MAX_df[Gemeinde_2020_500_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_1000_jan25_MAX = Gemeinde_2020_1000_MAX_df[Gemeinde_2020_1000_MAX_df['month_year'] == 'Jan-25']

In [ ]:
Gemeinde_2020_5_jan25_MAX.head()

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_by_risk = Gemeinde_2020_5_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_10_jan25_MAX_sa_by_risk = Gemeinde_2020_10_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_20_jan25_MAX_sa_by_risk = Gemeinde_2020_20_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_50_jan25_MAX_sa_by_risk = Gemeinde_2020_50_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_100_jan25_MAX_sa_by_risk = Gemeinde_2020_100_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_200_jan25_MAX_sa_by_risk = Gemeinde_2020_200_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_500_jan25_MAX_sa_by_risk = Gemeinde_2020_500_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_1000_jan25_MAX_sa_by_risk = Gemeinde_2020_1000_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_MAX"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'risk_level']).sum(numeric_only=True).reset_index()[['GEM_ID', 'risk_level', 'MAX_area_Jul16']]

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_MAX_sa_by_risk"] = grouped


In [ ]:
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

for rp in return_periods:
    jan25_name = f"Gemeinde_2020_{rp}_jan25_MAX_sa_by_risk"
    jul16_name = f"Gemeinde_2020_{rp}_Jul16_MAX_sa_by_risk"

    # Access the DataFrames from globals
    jan25_df = globals()[jan25_name]
    jul16_df = globals()[jul16_name]

    # Merge on ['GEM_ID', 'risk_level'], left join
    merged_df = jan25_df.merge(jul16_df, on=['GEM_ID', 'risk_level'], how='left')

    # Update the original jan25 variable with the merged result
    globals()[jan25_name] = merged_df


### Separate risk level further to high (>0.15 m), very high (>0.5 m) and extreme (> 1.5m)

In [ ]:
def very_high_risk(value):
    if value == '150' or value == '150p':
        return 'very_high'
    else:
        return 'other'

In [ ]:
for df in [Gemeinde_2020_5_jan25_MAX, Gemeinde_2020_10_jan25_MAX, Gemeinde_2020_20_jan25_MAX, Gemeinde_2020_50_jan25_MAX,
           Gemeinde_2020_100_jan25_MAX, Gemeinde_2020_200_jan25_MAX, Gemeinde_2020_500_jan25_MAX, Gemeinde_2020_1000_jan25_MAX]:
    df['very_risky'] = df.depth_cat.map(very_high_risk)

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_very_risky = Gemeinde_2020_5_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_10_jan25_MAX_sa_very_risky = Gemeinde_2020_10_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_20_jan25_MAX_sa_very_risky = Gemeinde_2020_20_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_50_jan25_MAX_sa_very_risky = Gemeinde_2020_50_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_100_jan25_MAX_sa_very_risky = Gemeinde_2020_100_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_200_jan25_MAX_sa_very_risky = Gemeinde_2020_200_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_500_jan25_MAX_sa_very_risky = Gemeinde_2020_500_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]
Gemeinde_2020_1000_jan25_MAX_sa_very_risky = Gemeinde_2020_1000_jan25_MAX.groupby(['GEM_ID', 'very_risky']).sum('MAX_area').reset_index()[['GEM_ID', 'very_risky', 'MAX_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the original DataFrame
    df = globals()[f"Gemeinde_2020_{rp}_jan25_MAX"]

    # Group by 'GEM_ID' and 'very_risky', sum 'MAX_area'
    grouped = df.groupby(['GEM_ID', 'very_risky']).sum(numeric_only=True).reset_index()[['GEM_ID', 'very_risky', 'MAX_area_Jul16']]
    grouped.columns = ['GEM_ID', 'risk_level', 'MAX_area_Jul16']

    # Assign to a new variable with the required name
    globals()[f"Gemeinde_2020_{rp}_Jul16_MAX_sa_very_risky"] = grouped


In [ ]:
Gemeinde_2020_5_Jul16_MAX_sa_very_risky.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_MAX']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'MAX_area']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'MAX_area']]

    globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_extreme'] = df_150p


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_MAX']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'MAX_area_Jul16']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'MAX_area_Jul16']]

    globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_extreme'] = df_150p


In [ ]:
for df in [Gemeinde_2020_5_jan25_MAX_sa_very_risky, Gemeinde_2020_10_jan25_MAX_sa_very_risky, Gemeinde_2020_20_jan25_MAX_sa_very_risky, Gemeinde_2020_50_jan25_MAX_sa_very_risky,
           Gemeinde_2020_100_jan25_MAX_sa_very_risky, Gemeinde_2020_200_jan25_MAX_sa_very_risky, Gemeinde_2020_500_jan25_MAX_sa_very_risky, Gemeinde_2020_1000_jan25_MAX_sa_very_risky]:
    df.columns = ['GEM_ID', 'risk_level', 'MAX_area']


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_risky_level'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_risky_level'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_MAX_sa_risky_level, Gemeinde_2020_10_jan25_MAX_sa_risky_level, Gemeinde_2020_20_jan25_MAX_sa_risky_level, Gemeinde_2020_50_jan25_MAX_sa_risky_level, \
           Gemeinde_2020_100_jan25_MAX_sa_risky_level, Gemeinde_2020_200_jan25_MAX_sa_risky_level, Gemeinde_2020_500_jan25_MAX_sa_risky_level, Gemeinde_2020_1000_jan25_MAX_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'MAX_area_by_level', 'MAX_area_Jul16_by_level']
    df['MAX_area_by_level_non_zero'] = df.MAX_area_by_level.replace(0, 1e-6)

In [ ]:
for df in [Gemeinde_2020_5_Jul16_MAX_sa_risky_level, Gemeinde_2020_10_Jul16_MAX_sa_risky_level, Gemeinde_2020_20_Jul16_MAX_sa_risky_level, Gemeinde_2020_50_Jul16_MAX_sa_risky_level, \
           Gemeinde_2020_100_Jul16_MAX_sa_risky_level, Gemeinde_2020_200_Jul16_MAX_sa_risky_level, Gemeinde_2020_500_Jul16_MAX_sa_risky_level, Gemeinde_2020_1000_Jul16_MAX_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'MAX_area_Jul16_by_level']
    df['MAX_area_Jul16_by_level_non_zero'] = df.MAX_area_Jul16_by_level.replace(0, 1e-6)

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_risky_level_df'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jul-16'][['GEM_ID', 'Settlement_Area']]
    settle_df.columns = ['GEM_ID', 'Settlement_Area_Jul16']

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_Jul16_MAX_sa_risky_level_df'] = merged_df

In [ ]:
for df in [Gemeinde_2020_5_jan25_MAX_sa_risky_level_df, Gemeinde_2020_10_jan25_MAX_sa_risky_level_df, Gemeinde_2020_20_jan25_MAX_sa_risky_level_df, Gemeinde_2020_50_jan25_MAX_sa_risky_level_df,
           Gemeinde_2020_100_jan25_MAX_sa_risky_level_df, Gemeinde_2020_200_jan25_MAX_sa_risky_level_df, Gemeinde_2020_500_jan25_MAX_sa_risky_level_df, Gemeinde_2020_1000_jan25_MAX_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['risk_level_area_perc'] = (df.MAX_area_by_level / df.Settlement_Area) * 100

In [ ]:
for df in [Gemeinde_2020_5_Jul16_MAX_sa_risky_level_df, Gemeinde_2020_10_Jul16_MAX_sa_risky_level_df, Gemeinde_2020_20_Jul16_MAX_sa_risky_level_df, Gemeinde_2020_50_Jul16_MAX_sa_risky_level_df,
           Gemeinde_2020_100_Jul16_MAX_sa_risky_level_df, Gemeinde_2020_200_Jul16_MAX_sa_risky_level_df, Gemeinde_2020_500_Jul16_MAX_sa_risky_level_df, Gemeinde_2020_1000_Jul16_MAX_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['risk_level_area_perc_Jul16'] = (df.MAX_area_Jul16_by_level / df.Settlement_Area_Jul16) * 100

In [ ]:
Gemeinde_2020_100_jan25_MAX_sa_risky_level_df[Gemeinde_2020_100_jan25_MAX_sa_risky_level_df.risk_level=='high_risk'].MAX_area_Jul16_by_level

In [ ]:
# I stored the processed dataframe as the pickle file to read directly so I can update the plots efficiently
Gemeinde_2020_100_jan25_MAX_sa_risky_level_df.to_pickle('Gemeinde_2020_100_jan25_MAX_sa_risky_level_df.pkl')

In [ ]:
Gemeinde_2020_100_Jul16_MAX_sa_risky_level_df.to_pickle('Gemeinde_2020_100_Jul16_MAX_sa_risky_level_df.pkl')

### Plotting prep for exposure plots at Gemeinden level

In [ ]:
dfs_risk_level_jan25_MAX = [Gemeinde_2020_5_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_10_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_20_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_50_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_100_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_200_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_500_jan25_MAX_sa_risky_level_df,
                      Gemeinde_2020_1000_jan25_MAX_sa_risky_level_df]

In [ ]:
# Create labels for each scenario
labels = ['5-yr', '10-yr', '20-yr','50-yr', '100-yr', '200-yr', '500-yr', '1000-yr']

# Add scenario labels and combine into one DataFrame
for df, label in zip(dfs_risk_level_jan25_MAX, labels):
    df['return_period'] = label

df_all_risk_level_jan25_MAX = pd.concat(dfs_risk_level_jan25_MAX, ignore_index=True)

### Plot: Separate risk level further to high (>0.15 m), very high (>0.5 m) and extreme (> 1.5m)

In [ ]:
# Setup
sns.set(style="whitegrid")
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_all_period_plots/final_adjustment"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_land_share_Jan25_by_scenario_strip.png"
full_path = os.path.join(save_path, file_name)

# Style and color
red_shades = ['#ff9999', '#ff1a1a', '#800000']
risk_order = ['high_risk', 'very_high', 'extreme']
risk_label_map = {
    'high_risk': 'Flood depth > 0.15 m',
    'very_high': 'Flood depth > 0.5 m',
    'extreme': 'Flood depth > 1.5 m'
}
color_map = dict(zip(risk_order, red_shades))
# Remove '-yr' suffix and convert to numeric
df_all_risk_level_jan25_MAX['return_period_num'] = df_all_risk_level_jan25_MAX['return_period'].str.replace('-yr', '', regex=False).astype(int)


# Create 1x3 subplot
fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Plot each risk level
for idx, risk in enumerate(risk_order):
    ax = axs[idx]
    subset = df_all_risk_level_jan25_MAX[df_all_risk_level_jan25_MAX['risk_level'] == risk]

    sns.stripplot(
        data=subset,
        x='return_period_num',
        y='risk_level_area_perc',
        color=color_map[risk],
        s=2,
        alpha=0.1,
        ax=ax,
        zorder = -1
    )

    # Calculate and draw percentiles
    grouped = subset.groupby('return_period')['risk_level_area_perc']
    for rp, group in grouped:
        percentiles = {
            '50': group.median(),
            '75': group.quantile(0.75),
            '90': group.quantile(0.90)
        }
        xpos = list(subset['return_period'].unique()).index(rp)

        # Use different styles
        ax.hlines(percentiles['50'], xpos - 0.15, xpos + 0.15, color='black', linewidth=1.5, linestyles='solid')
        ax.hlines(percentiles['75'], xpos - 0.2, xpos + 0.2, color='black', linewidth=1.2, linestyles='dashed')
        ax.hlines(percentiles['90'], xpos - 0.3, xpos + 0.3, color='black', linewidth=1, linestyles='dotted')

    # Formatting
    ax.set_title(risk_label_map[risk], fontsize=18)
    ax.set_xlabel('')
    if idx == 0:
        ax.set_ylabel('Exposure share (%)', fontsize=16)
    else:
        ax.set_ylabel('')
    ax.tick_params(axis='x', rotation=45, labelsize=14)
    ax.tick_params(axis='y', labelsize=14)

# Custom legend for percentiles
legend_lines = [
    Line2D([0], [0], color='black', linewidth=1.5, linestyle='solid', label='50th percentile'),
    Line2D([0], [0], color='black', linewidth=1.2, linestyle='dashed', label='75th percentile'),
    Line2D([0], [0], color='black', linewidth=1, linestyle='dotted', label='90th percentile')
]

# Add a single legend for the figure, centered below subplots
fig.legend(
    handles=legend_lines,
    # title='Percentile',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.15),  # Adjust vertical position
    ncol=len(legend_lines),
    fontsize=16,
    title_fontsize=18
)

# X-axis label for all subplots
fig.supxlabel("Flood return periods (years)", fontsize=18)

plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()

### Pull out summary statistics from the plots above for the table in the appendix

In [ ]:
# Calculate and print percentiles
risk_order = ['high_risk', 'very_high', 'extreme']
risk_label_map = {
    'high_risk': 'Flood Depth > 0.15 m',
    'very_high': 'Flood Depth > 0.5 m',
    'extreme': 'Flood Depth > 1.5 m'
}

# Function to extract numeric return period
def extract_rp_value(rp_label):
    match = re.search(r'(\d+)', rp_label)
    return int(match.group(1)) if match else float('inf')

# Calculate and print percentiles in numeric order
print("Percentiles (Jan-25):\n")
for risk in risk_order:
    subset = df_all_risk_level_jan25_MAX[df_all_risk_level_jan25_MAX['risk_level'] == risk]
    print(f"\nRisk Level: {risk_label_map[risk]}")
    grouped = subset.groupby('return_period')['risk_level_area_perc']

    # Sort return periods numerically
    sorted_rps = sorted(grouped.groups.keys(), key=extract_rp_value)

    for rp in sorted_rps:
        group = grouped.get_group(rp)
        percentiles = {
            '50th': group.median(),
            '75th': group.quantile(0.75),
            '90th': group.quantile(0.90)
        }
        print(f"  Return Period {rp}: 50th={percentiles['50th']:.2f}%, 75th={percentiles['75th']:.2f}%, 90th={percentiles['90th']:.2f}%")



### Because we only map high risk (>0.15 m) exposure, we would need to go back a little bit and re-processing the data: Create risk level - low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
def risk_level(value):
    if value == '0' or value == '015':
        return 'low_risk'
    else:
        return 'high_risk'

In [ ]:
for df in [Gemeinde_2020_5_MAX_df, Gemeinde_2020_10_MAX_df, Gemeinde_2020_20_MAX_df, Gemeinde_2020_50_MAX_df,
           Gemeinde_2020_100_MAX_df, Gemeinde_2020_200_MAX_df, Gemeinde_2020_500_MAX_df, Gemeinde_2020_1000_MAX_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Gemeinde_2020_5_MAX_df.head()

In [ ]:
Gemeinde_2020_5_jan25_MAX = Gemeinde_2020_5_MAX_df[Gemeinde_2020_5_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_10_jan25_MAX = Gemeinde_2020_10_MAX_df[Gemeinde_2020_10_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_20_jan25_MAX = Gemeinde_2020_20_MAX_df[Gemeinde_2020_20_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_50_jan25_MAX = Gemeinde_2020_50_MAX_df[Gemeinde_2020_50_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_100_jan25_MAX = Gemeinde_2020_100_MAX_df[Gemeinde_2020_100_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_200_jan25_MAX = Gemeinde_2020_200_MAX_df[Gemeinde_2020_200_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_500_jan25_MAX = Gemeinde_2020_500_MAX_df[Gemeinde_2020_500_MAX_df['month_year'] == 'Jan-25']
Gemeinde_2020_1000_jan25_MAX = Gemeinde_2020_1000_MAX_df[Gemeinde_2020_1000_MAX_df['month_year'] == 'Jan-25']

In [ ]:
Gemeinde_2020_5_jan25_MAX.head()

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_by_risk = Gemeinde_2020_5_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_10_jan25_MAX_sa_by_risk = Gemeinde_2020_10_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_20_jan25_MAX_sa_by_risk = Gemeinde_2020_20_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_50_jan25_MAX_sa_by_risk = Gemeinde_2020_50_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_100_jan25_MAX_sa_by_risk = Gemeinde_2020_100_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_200_jan25_MAX_sa_by_risk = Gemeinde_2020_200_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_500_jan25_MAX_sa_by_risk = Gemeinde_2020_500_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]
Gemeinde_2020_1000_jan25_MAX_sa_by_risk = Gemeinde_2020_1000_jan25_MAX.groupby(['GEM_ID', 'risk_level']).sum('MAX_area').reset_index()[['GEM_ID', 'risk_level', 'MAX_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_MAX"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'risk_level']).sum(numeric_only=True).reset_index()[['GEM_ID', 'risk_level', 'MAX_area_Jul16']]

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_MAX_sa_by_risk"] = grouped


In [ ]:
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

for rp in return_periods:
    jan25_name = f"Gemeinde_2020_{rp}_jan25_MAX_sa_by_risk"
    jul16_name = f"Gemeinde_2020_{rp}_Jul16_MAX_sa_by_risk"

    # Access the DataFrames from globals
    jan25_df = globals()[jan25_name]
    jul16_df = globals()[jul16_name]

    # Merge on ['GEM_ID', 'risk_level'], left join
    merged_df = jan25_df.merge(jul16_df, on=['GEM_ID', 'risk_level'], how='left')

    # Update the original jan25 variable with the merged result
    globals()[jan25_name] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_MAX_sa_by_risk, Gemeinde_2020_10_jan25_MAX_sa_by_risk, Gemeinde_2020_20_jan25_MAX_sa_by_risk, Gemeinde_2020_50_jan25_MAX_sa_by_risk,
           Gemeinde_2020_100_jan25_MAX_sa_by_risk, Gemeinde_2020_200_jan25_MAX_sa_by_risk, Gemeinde_2020_500_jan25_MAX_sa_by_risk, Gemeinde_2020_1000_jan25_MAX_sa_by_risk]:
    df.columns = ['GEM_ID', 'risk_level', 'Max_area_by_risk', 'Max_area_Jul16_by_risk']
    df['Max_area_by_risk_non_zero'] = df.Max_area_by_risk.replace(0, 1e-6)
    df['Max_area_Jul16_by_risk_non_zero'] = df.Max_area_Jul16_by_risk.replace(0, 1e-6)
    df['pct_growth_by_risk'] = ((df['Max_area_by_risk'] - df['Max_area_Jul16_by_risk'])/df['Max_area_Jul16_by_risk_non_zero']) * 100

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_by_risk']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_by_risk_df'] = merged_df


In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_by_risk_df.head()

In [ ]:
for df in [Gemeinde_2020_5_jan25_MAX_sa_by_risk_df, Gemeinde_2020_10_jan25_MAX_sa_by_risk_df, Gemeinde_2020_20_jan25_MAX_sa_by_risk_df, Gemeinde_2020_50_jan25_MAX_sa_by_risk_df,
           Gemeinde_2020_100_jan25_MAX_sa_by_risk_df, Gemeinde_2020_200_jan25_MAX_sa_by_risk_df, Gemeinde_2020_500_jan25_MAX_sa_by_risk_df, Gemeinde_2020_1000_jan25_MAX_sa_by_risk_df]:

    risk_order = ['low_risk', 'high_risk']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['risk_level_area_perc'] = (df.Max_area_by_risk / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_by_risk_df.head()

In [ ]:
dfs_MAX = [Gemeinde_2020_5_MAX_df, Gemeinde_2020_10_MAX_df, Gemeinde_2020_20_MAX_df, Gemeinde_2020_50_MAX_df,
       Gemeinde_2020_100_MAX_df, Gemeinde_2020_200_MAX_df, Gemeinde_2020_500_MAX_df, Gemeinde_2020_1000_MAX_df]

# Create labels for each scenario
labels = ['5-yr', '10-yr', '20-yr','50-yr', '100-yr', '200-yr', '500-yr', '1000-yr']

In [ ]:
# Add scenario labels and combine into one DataFrame
for df, label in zip(dfs_MAX, labels):
    df['scenario'] = label

df_all_MAX = pd.concat(dfs_MAX, ignore_index=True)

# Optional: sort month_year for clean x-axis
df_all_MAX['month_year'] = pd.Categorical(
    df_all_MAX['month_year'],
    ordered=True,
    categories=sorted(df_all_MAX['month_year'].unique(), key=lambda x: pd.to_datetime('01-' + x, format='%d-%b-%y'))
)

In [ ]:
dfs_risk_jan25_MAX = [Gemeinde_2020_5_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_10_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_20_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_50_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_100_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_200_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_500_jan25_MAX_sa_by_risk_df,
                      Gemeinde_2020_1000_jan25_MAX_sa_by_risk_df]

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_by_risk_df.head()

In [ ]:
# Add scenario labels and combine into one DataFrame
for df, label in zip(dfs_risk_jan25_MAX, labels):
    df['return_period'] = label

df_all_risk_jan25_MAX = pd.concat(dfs_risk_jan25_MAX, ignore_index=True)

In [ ]:
df_all_risk_jan25_MAX.columns

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_by_risk_df.head()

In [ ]:
# I stored the processed dataframe as the pickle file to read directly so I can update the plots efficiently
Gemeinde_2020_100_jan25_MAX_sa_by_risk_df.to_pickle('Gemeinde_2020_100_jan25_MAX_sa_by_risk_df.pkl')

In [ ]:
Gemeinde_2020_100_jan25_MAX_sa_by_risk_df = pd.read_pickle('Gemeinde_2020_100_jan25_MAX_sa_by_risk_df.pkl')

### Create the maps: combined flooding exposure

In [ ]:
# Print out basemap options to choose from
print(ctx.providers.keys())

In [ ]:
Gemeinde_2020_5_jan25_MAX_sa_high_risk_df = Gemeinde_2020_5_jan25_MAX_sa_by_risk_df[Gemeinde_2020_5_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_10_jan25_MAX_sa_high_risk_df = Gemeinde_2020_10_jan25_MAX_sa_by_risk_df[Gemeinde_2020_10_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_20_jan25_MAX_sa_high_risk_df = Gemeinde_2020_20_jan25_MAX_sa_by_risk_df[Gemeinde_2020_200_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_50_jan25_MAX_sa_high_risk_df = Gemeinde_2020_50_jan25_MAX_sa_by_risk_df[Gemeinde_2020_50_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_100_jan25_MAX_sa_high_risk_df = Gemeinde_2020_100_jan25_MAX_sa_by_risk_df[Gemeinde_2020_100_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_200_jan25_MAX_sa_high_risk_df = Gemeinde_2020_200_jan25_MAX_sa_by_risk_df[Gemeinde_2020_200_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_500_jan25_MAX_sa_high_risk_df = Gemeinde_2020_500_jan25_MAX_sa_by_risk_df[Gemeinde_2020_500_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_1000_jan25_MAX_sa_high_risk_df = Gemeinde_2020_1000_jan25_MAX_sa_by_risk_df[Gemeinde_2020_1000_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    jan25_high_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_high_risk_df']

    merged_gdf = admin3_boundary.merge(jan25_high_risk_df, left_on='OBJID', right_on='GEM_ID', how='right')

    globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_high_risk_gdf'] = merged_gdf

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_all_period_maps/pct_growth_maps/final_adjustments"
os.makedirs(save_path, exist_ok=True)

for y in years:
    # Filter and project the data
    gdf = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_high_risk_gdf'].copy()
    gdf = gdf.to_crs(epsg=3857)

    # Mask 0 values for transparency
    gdf['masked_pct_growth'] = gdf['pct_growth_by_risk'].replace(0, np.nan)

    gemeinde_plot2 = gdf[gdf['masked_pct_growth'].notna()].copy()

    # Define custom breaks
    bins = [0, 10, 25, 50, 75, float('inf')]
    labels = [0, 1, 2, 3, 4]  # For matching colormap index

    # Apply binning
    gemeinde_plot2['growth_bin'] = pd.cut(
        gemeinde_plot2['masked_pct_growth'],
        bins=bins,
        labels=labels,
        include_lowest=True
    ).astype('Int64')


    # Red colormap
    cmap_steps = 5
    colormap = cm.Reds(np.linspace(0.2, 0.95, cmap_steps))

    # Assign colors based on bins
    def get_growth_color(row):
        bin_idx = row['growth_bin']
        if pd.isna(bin_idx):
            return "#d3d3d3"  # Light grey for NaNs
        try:
            bin_idx = int(bin_idx)
            return to_hex(colormap[bin_idx])
        except Exception as e:
            print(f"Error on row {row.name}: {e}")
            return "#000000"

    gemeinde_plot2['color'] = gemeinde_plot2.apply(get_growth_color, axis=1)

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 10))
    gemeinde_plot2.plot(ax=ax, color=gemeinde_plot2['color'], edgecolor=None, linewidth=0.2)

    # Gemeinde boundary
    gdf.boundary.plot(ax=ax, color='grey', linewidth=0.2)

    # Basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gdf.crs)

    # Title and cleanup
    # ax.set_title(f"%Growth of Exposure to Max Flood Depths Greater Than 0.15 m, Gemeinden level — {y}-year return period", fontsize=12)
    ax.axis('off')

    # North arrow and scale bar
    add_north_arrow(ax, scale=0.5, xlim_pos=0.925, ylim_pos=0.85, color='black', text_scaler=4, text_yT=-1.25)
    scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
    ax.add_artist(scale1)

    legend_labels = [
        "0% ~ 10%",
        "10% ~ 25%",
        "25% ~ 50%",
        "50% ~ 75%",
        "75% ~ 100%+"
    ]

    circles = []
    for i, label in enumerate(legend_labels):
        circle_marker = Line2D(
            [0], [0],
            marker='o',
            color='None',
            markerfacecolor=to_hex(colormap[i]),
            markeredgecolor='None',
            markersize=10,
            label=label
        )
        circles.append(circle_marker)


    legend1 = ax.legend(
        handles=circles,
        title="Exposure growth",
        loc='upper left',
        # bbox_to_anchor=(-0.02, 1.05),
        fontsize=12,
        title_fontsize=14
    )

    ax.add_artist(legend1)


    # Save figure (both PNG and SVG)
    file_name_base = f"customized_break_gemeinde_exposed_high_risk_pct_growth_Jan25_{y}yr"
    png_path = os.path.join(save_path, file_name_base + ".png")
    svg_path = os.path.join(save_path, file_name_base + ".svg")

    plt.tight_layout()
    # fig.savefig(png_path, dpi=300, bbox_inches="tight")       # High-res PNG
    fig.savefig(svg_path, format="svg", bbox_inches="tight")  # Vector SVG
    plt.show()
    # plt.close(fig)  # close after saving to free memory


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_all_period_maps/final_adjusted_maps"
os.makedirs(save_path, exist_ok=True)

# Define your custom breaks
custom_bins = [8, 20, 40, 72, 100.1]

# Create a colormap with one fewer color than the number of bins
cmap = 'Reds'

for y in years:
    gdf = globals()[f'Gemeinde_2020_{y}_jan25_MAX_sa_high_risk_gdf'].copy()
    gdf = gdf.to_crs(epsg=3857)

    # Mask 0 values for transparency
    gdf['masked_share'] = gdf['risk_level_area_perc'].replace(0, np.nan)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))
    plot = gdf.plot(
        ax=ax,
        column='masked_share',
        cmap=cmap,
        edgecolor=None,
        linewidth=0.5,
        legend=True,
        scheme='user_defined',  # specify custom scheme
        classification_kwds={'bins': custom_bins},
        legend_kwds={
            'loc': 'upper left',
            'fmt': "{:.0f}",
            'title': "Exposed \nsettlement (%)",
            'title_fontsize': 14,   # larger legend title
            'fontsize': 12          # larger legend labels
        }
    )

    # Overlay geometry boundaries
    gdf.boundary.plot(
        ax=ax,
        color='grey',
        linewidth=0.05
    )

    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gdf.crs)

    # Remove title
    # ax.set_title(...)   <- deleted
    ax.axis('off')

    # Add north arrow and scale bar
    add_north_arrow(ax, scale=0.75, xlim_pos=0.9025, ylim_pos=0.9, color='black', text_scaler=4, text_yT=-1.25)
    scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
    ax.add_artist(scale1)

    # Save
    file_name_base = f"custom_break_gemeinde_exposed_high_risk_land_share_Jan25_{y}yr"
    plt.tight_layout()

    # png_path = os.path.join(save_path, file_name_base + ".png")
    svg_path = os.path.join(save_path, file_name_base + ".svg")

    plt.tight_layout()
    # fig.savefig(png_path, dpi=300, bbox_inches="tight")       # High-res PNG
    fig.savefig(svg_path, format="svg", bbox_inches="tight")  # Vector SVG
    plt.show()


## Coastal flooding undefended

### data preparation

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
cu_columns = new_cols[:7] + [col for col in new_cols if col.startswith('SA_CU')]

years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_CU'] = globals()[f'Gemeinde_2020_{y}_raw'][cu_columns]


In [ ]:
years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_CU_long'] = globals()[f'Gemeinde_2020_{y}_CU'].melt(
        id_vars=new_cols[:7], var_name='month_year_depth', value_name='CU_area'
    )

In [ ]:
Gemeinde_2020_5_CU_long.head()

In [ ]:
for df in [Gemeinde_2020_5_CU_long, Gemeinde_2020_10_CU_long, Gemeinde_2020_20_CU_long, Gemeinde_2020_50_CU_long,
           Gemeinde_2020_100_CU_long, Gemeinde_2020_200_CU_long, Gemeinde_2020_500_CU_long, Gemeinde_2020_1000_CU_long]:

    # Extract depth category (e.g., '0', '015', '050', '150', '150p') and month-year (e.g., 'Jul-16')
    df[['depth_cat', 'month_year']] = df['month_year_depth'].str.extract(r'_([0-9]{1,4}p?)_([A-Za-z]{3}-\d{2})$')

In [ ]:
Gemeinde_2020_5_CU_long.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    long_df = globals()[f'Gemeinde_2020_{y}_CU_long']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][['GEM_ID', 'month_year', 'Settlement_Area']]

    merged_df = pd.merge(long_df, settle_df, on=['GEM_ID', 'month_year'], how='left')

    globals()[f'Gemeinde_2020_{y}_CU_df'] = merged_df


In [ ]:
Gemeinde_2020_5_CU_df.head()

In [ ]:
for df in [Gemeinde_2020_5_CU_df, Gemeinde_2020_10_CU_df, Gemeinde_2020_20_CU_df, Gemeinde_2020_50_CU_df,
           Gemeinde_2020_100_CU_df, Gemeinde_2020_200_CU_df, Gemeinde_2020_500_CU_df, Gemeinde_2020_1000_CU_df]:
    df['depth_cat_area_perc_cu'] = (df.CU_area / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_CU_baseline = Gemeinde_2020_5_CU_df[Gemeinde_2020_5_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_10_CU_baseline = Gemeinde_2020_10_CU_df[Gemeinde_2020_10_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_20_CU_baseline = Gemeinde_2020_20_CU_df[Gemeinde_2020_20_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_50_CU_baseline = Gemeinde_2020_50_CU_df[Gemeinde_2020_50_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_100_CU_baseline = Gemeinde_2020_100_CU_df[Gemeinde_2020_100_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_200_CU_baseline = Gemeinde_2020_200_CU_df[Gemeinde_2020_200_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_500_CU_baseline = Gemeinde_2020_500_CU_df[Gemeinde_2020_500_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]
Gemeinde_2020_1000_CU_baseline = Gemeinde_2020_1000_CU_df[Gemeinde_2020_1000_CU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'CU_area']]

In [ ]:
for y in [5, 10, 20, 50, 100, 200, 500, 1000]:
    df_name = f'Gemeinde_2020_{y}_CU_baseline'
    df = globals()[df_name]
    globals()[df_name] = df.rename(columns={'CU_area': 'CU_area_Jul16'})

In [ ]:
for df_name in ['Gemeinde_2020_5_CU_baseline', 'Gemeinde_2020_10_CU_baseline', 'Gemeinde_2020_20_CU_baseline', 'Gemeinde_2020_50_CU_baseline',
                'Gemeinde_2020_100_CU_baseline', 'Gemeinde_2020_200_CU_baseline', 'Gemeinde_2020_500_CU_baseline', 'Gemeinde_2020_1000_CU_baseline']:
    df = globals()[df_name]
    df = df.rename(columns={'CU_area': 'CU_area_Jul16'})  # Rename first
    df['CU_area_Jul16_non_zero'] = df['CU_area_Jul16'].replace(0, 1e-6)  # Replace zeros
    globals()[df_name] = df

In [ ]:
Gemeinde_2020_5_CU_baseline.head()

In [ ]:
for return_period in [5, 10, 20, 50, 100, 200, 500, 1000]:
    orig_df = globals()[f'Gemeinde_2020_{return_period}_CU_df']
    baseline_df = globals()[f'Gemeinde_2020_{return_period}_CU_baseline']
    merged_df = orig_df.merge(baseline_df, on=['GEM_ID', 'depth_cat'], how='left')
    globals()[f'Gemeinde_2020_{return_period}_CU_df'] = merged_df

In [ ]:
Gemeinde_2020_5_CU_df.head()

In [ ]:
for df in [Gemeinde_2020_5_CU_df, Gemeinde_2020_10_CU_df, Gemeinde_2020_20_CU_df, Gemeinde_2020_50_CU_df,
           Gemeinde_2020_100_CU_df, Gemeinde_2020_200_CU_df, Gemeinde_2020_500_CU_df, Gemeinde_2020_1000_CU_df]:
    df['CU_area_non_zero'] = df.CU_area.replace(0, 1e-6)
    df['pct_growth'] = (df['CU_area_non_zero'] - df['CU_area_Jul16_non_zero']) / df['CU_area_Jul16_non_zero'] * 100

In [ ]:
def risk_level(value):
    if value == '0' or value == '015':
        return 'low_risk'
    else:
        return 'high_risk'

In [ ]:
for df in [Gemeinde_2020_5_CU_df, Gemeinde_2020_10_CU_df, Gemeinde_2020_20_CU_df, Gemeinde_2020_50_CU_df,
           Gemeinde_2020_100_CU_df, Gemeinde_2020_200_CU_df, Gemeinde_2020_500_CU_df, Gemeinde_2020_1000_CU_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Gemeinde_2020_5_CU_df.head()

In [ ]:
Gemeinde_2020_5_jan25_CU = Gemeinde_2020_5_CU_df[Gemeinde_2020_5_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_10_jan25_CU = Gemeinde_2020_10_CU_df[Gemeinde_2020_10_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_20_jan25_CU = Gemeinde_2020_20_CU_df[Gemeinde_2020_20_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_50_jan25_CU = Gemeinde_2020_50_CU_df[Gemeinde_2020_50_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_100_jan25_CU = Gemeinde_2020_100_CU_df[Gemeinde_2020_100_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_200_jan25_CU = Gemeinde_2020_200_CU_df[Gemeinde_2020_200_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_500_jan25_CU = Gemeinde_2020_500_CU_df[Gemeinde_2020_500_CU_df['month_year'] == 'Jan-25']
Gemeinde_2020_1000_jan25_CU = Gemeinde_2020_1000_CU_df[Gemeinde_2020_1000_CU_df['month_year'] == 'Jan-25']

In [ ]:
Gemeinde_2020_5_jan25_CU.head()

In [ ]:
Gemeinde_2020_5_jan25_CU_sa_by_risk = Gemeinde_2020_5_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_10_jan25_CU_sa_by_risk = Gemeinde_2020_10_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_20_jan25_CU_sa_by_risk = Gemeinde_2020_20_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_50_jan25_CU_sa_by_risk = Gemeinde_2020_50_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_100_jan25_CU_sa_by_risk = Gemeinde_2020_100_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_200_jan25_CU_sa_by_risk = Gemeinde_2020_200_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_500_jan25_CU_sa_by_risk = Gemeinde_2020_500_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]
Gemeinde_2020_1000_jan25_CU_sa_by_risk = Gemeinde_2020_1000_jan25_CU.groupby(['GEM_ID', 'risk_level']).sum('CU_area').reset_index()[['GEM_ID', 'risk_level', 'CU_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_CU"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'risk_level']).sum(numeric_only=True).reset_index()[['GEM_ID', 'risk_level', 'CU_area_Jul16']]

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_CU_sa_by_risk"] = grouped


### Continue to create dataframe separating the flood depth with only two categories: low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
for df in [Gemeinde_2020_5_jan25_CU_sa_by_risk, Gemeinde_2020_10_jan25_CU_sa_by_risk, Gemeinde_2020_20_jan25_CU_sa_by_risk, Gemeinde_2020_50_jan25_CU_sa_by_risk,
           Gemeinde_2020_100_jan25_CU_sa_by_risk, Gemeinde_2020_200_jan25_CU_sa_by_risk, Gemeinde_2020_500_jan25_CU_sa_by_risk, Gemeinde_2020_1000_jan25_CU_sa_by_risk]:
    df.columns = ['GEM_ID', 'risk_level', 'CU_area_by_risk']
    df['CU_area_by_risk_non_zero'] = df.CU_area_by_risk.replace(0, 1e-6)

In [ ]:
Gemeinde_2020_5_jan25_CU_sa_by_risk.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_by_risk']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_by_risk_df'] = merged_df


In [ ]:
# Gemeinde_2020_5_jan25_MAX_sa_by_risk_df.head()

In [ ]:
for df in [Gemeinde_2020_5_jan25_CU_sa_by_risk_df, Gemeinde_2020_10_jan25_CU_sa_by_risk_df, Gemeinde_2020_20_jan25_CU_sa_by_risk_df, Gemeinde_2020_50_jan25_CU_sa_by_risk_df,
           Gemeinde_2020_100_jan25_CU_sa_by_risk_df, Gemeinde_2020_200_jan25_CU_sa_by_risk_df, Gemeinde_2020_500_jan25_CU_sa_by_risk_df, Gemeinde_2020_1000_jan25_CU_sa_by_risk_df]:

    risk_order = ['low_risk', 'high_risk']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['risk_level_area_perc_cu'] = (df.CU_area_by_risk / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_jan25_CU_sa_by_risk_df.head()

In [ ]:
#Store this to a temporary pickle file first and read it later
Gemeinde_2020_100_jan25_CU_sa_by_risk_df.to_pickle("Gemeinde_2020_100_jan25_CU_sa_by_risk_df.pkl")

### **Please run the "data preparation" under Coastal flooding undefended again** to make this section work properly: Separate risk level further to high (>0.15 m), very high (>0.5 m) and extreme (> 1.5m)

In [ ]:
def very_high_risk(value):
    if value == '150' or value == '150p':
        return 'very_high'
    else:
        return 'other'

In [ ]:
for df in [Gemeinde_2020_5_jan25_CU, Gemeinde_2020_10_jan25_CU, Gemeinde_2020_20_jan25_CU, Gemeinde_2020_50_jan25_CU,
           Gemeinde_2020_100_jan25_CU, Gemeinde_2020_200_jan25_CU, Gemeinde_2020_500_jan25_CU, Gemeinde_2020_1000_jan25_CU]:
    df['very_risky'] = df.depth_cat.map(very_high_risk)

In [ ]:
Gemeinde_2020_5_jan25_CU_sa_very_risky = Gemeinde_2020_5_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_10_jan25_CU_sa_very_risky = Gemeinde_2020_10_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_20_jan25_CU_sa_very_risky = Gemeinde_2020_20_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_50_jan25_CU_sa_very_risky = Gemeinde_2020_50_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_100_jan25_CU_sa_very_risky = Gemeinde_2020_100_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_200_jan25_CU_sa_very_risky = Gemeinde_2020_200_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_500_jan25_CU_sa_very_risky = Gemeinde_2020_500_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]
Gemeinde_2020_1000_jan25_CU_sa_very_risky = Gemeinde_2020_1000_jan25_CU.groupby(['GEM_ID', 'very_risky']).sum('CU_area').reset_index()[['GEM_ID', 'very_risky', 'CU_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_CU"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'very_risky']).sum(numeric_only=True).reset_index()[['GEM_ID', 'very_risky', 'CU_area_Jul16']]
    grouped.columns = ['GEM_ID', 'risk_level', 'CU_area_Jul16']

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_CU_sa_very_risky"] = grouped


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_CU']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'CU_area']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'CU_area']]

    globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_extreme'] = df_150p


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_CU']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'CU_area_Jul16']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'CU_area_Jul16']]

    globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_extreme'] = df_150p


In [ ]:
for df in [Gemeinde_2020_5_jan25_CU_sa_very_risky, Gemeinde_2020_10_jan25_CU_sa_very_risky, Gemeinde_2020_20_jan25_CU_sa_very_risky, Gemeinde_2020_50_jan25_CU_sa_very_risky,
           Gemeinde_2020_100_jan25_CU_sa_very_risky, Gemeinde_2020_200_jan25_CU_sa_very_risky, Gemeinde_2020_500_jan25_CU_sa_very_risky, Gemeinde_2020_1000_jan25_CU_sa_very_risky]:
    df.columns = ['GEM_ID', 'risk_level', 'CU_area']


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_risky_level'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_risky_level'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_CU_sa_risky_level, Gemeinde_2020_10_jan25_CU_sa_risky_level, Gemeinde_2020_20_jan25_CU_sa_risky_level, Gemeinde_2020_50_jan25_CU_sa_risky_level, \
           Gemeinde_2020_100_jan25_CU_sa_risky_level, Gemeinde_2020_200_jan25_CU_sa_risky_level, Gemeinde_2020_500_jan25_CU_sa_risky_level, Gemeinde_2020_1000_jan25_CU_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'CU_area_by_level']
    df['CU_area_by_level_non_zero'] = df.CU_area_by_level.replace(0, 1e-6)

In [ ]:
for df in [Gemeinde_2020_5_Jul16_CU_sa_risky_level, Gemeinde_2020_10_Jul16_CU_sa_risky_level, Gemeinde_2020_20_Jul16_CU_sa_risky_level, Gemeinde_2020_50_Jul16_CU_sa_risky_level, \
           Gemeinde_2020_100_Jul16_CU_sa_risky_level, Gemeinde_2020_200_Jul16_CU_sa_risky_level, Gemeinde_2020_500_Jul16_CU_sa_risky_level, Gemeinde_2020_1000_Jul16_CU_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'CU_area_Jul16_by_level']
    df['CU_area__Jul16_by_level_non_zero'] = df.CU_area_Jul16_by_level.replace(0, 1e-6)

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_CU_sa_risky_level_df'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jul-16'][['GEM_ID', 'Settlement_Area']]
    settle_df.columns = ['GEM_ID', 'Settlement_Area_Jul16']

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_Jul16_CU_sa_risky_level_df'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_CU_sa_risky_level_df, Gemeinde_2020_10_jan25_CU_sa_risky_level_df, Gemeinde_2020_20_jan25_CU_sa_risky_level_df, Gemeinde_2020_50_jan25_CU_sa_risky_level_df,
           Gemeinde_2020_100_jan25_CU_sa_risky_level_df, Gemeinde_2020_200_jan25_CU_sa_risky_level_df, Gemeinde_2020_500_jan25_CU_sa_risky_level_df, Gemeinde_2020_1000_jan25_CU_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['CU_risk_level_area_perc'] = (df.CU_area_by_level / df.Settlement_Area) * 100

In [ ]:
for df in [Gemeinde_2020_5_Jul16_CU_sa_risky_level_df, Gemeinde_2020_10_Jul16_CU_sa_risky_level_df, Gemeinde_2020_20_Jul16_CU_sa_risky_level_df, Gemeinde_2020_50_Jul16_CU_sa_risky_level_df, \
           Gemeinde_2020_100_Jul16_CU_sa_risky_level_df, Gemeinde_2020_200_Jul16_CU_sa_risky_level_df, Gemeinde_2020_500_Jul16_CU_sa_risky_level_df, Gemeinde_2020_1000_Jul16_CU_sa_risky_level_df]:
    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['CU_risk_level_area_perc_Jul16'] = (df.CU_area_Jul16_by_level / df.Settlement_Area_Jul16) * 100

In [ ]:
Gemeinde_2020_5_Jul16_CU_sa_risky_level_df.CU_area_Jul16_by_level.describe()

In [ ]:
#Store the processed data to pickle file first and read it later
Gemeinde_2020_100_jan25_CU_sa_risky_level_df.to_pickle("Gemeinde_2020_100_jan25_CU_sa_risky_level_df.pkl")

In [ ]:
#Store the processed data to pickle file first and read it later
Gemeinde_2020_100_Jul16_CU_sa_risky_level_df.to_pickle("Gemeinde_2020_100_Jul16_CU_sa_risky_level_df.pkl")

## Fluvial flooding undefended

### data preparation

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
fu_columns = new_cols[:7] + [col for col in new_cols if col.startswith('SA_FU')]

years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_FU'] = globals()[f'Gemeinde_2020_{y}_raw'][fu_columns]


In [ ]:
years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_FU_long'] = globals()[f'Gemeinde_2020_{y}_FU'].melt(
        id_vars=new_cols[:7], var_name='month_year_depth', value_name='FU_area'
    )

In [ ]:
Gemeinde_2020_5_FU_long.head()

In [ ]:
for df in [Gemeinde_2020_5_FU_long, Gemeinde_2020_10_FU_long, Gemeinde_2020_20_FU_long, Gemeinde_2020_50_FU_long,
           Gemeinde_2020_100_FU_long, Gemeinde_2020_200_FU_long, Gemeinde_2020_500_FU_long, Gemeinde_2020_1000_FU_long]:

    # Extract depth category (e.g., '0', '015', '050', '150', '150p') and month-year (e.g., 'Jul-16')
    df[['depth_cat', 'month_year']] = df['month_year_depth'].str.extract(r'_([0-9]{1,4}p?)_([A-Za-z]{3}-\d{2})$')

In [ ]:
Gemeinde_2020_5_FU_long.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    long_df = globals()[f'Gemeinde_2020_{y}_FU_long']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][['GEM_ID', 'month_year', 'Settlement_Area']]

    merged_df = pd.merge(long_df, settle_df, on=['GEM_ID', 'month_year'], how='left')

    globals()[f'Gemeinde_2020_{y}_FU_df'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_FU_df, Gemeinde_2020_10_FU_df, Gemeinde_2020_20_FU_df, Gemeinde_2020_50_FU_df,
           Gemeinde_2020_100_FU_df, Gemeinde_2020_200_FU_df, Gemeinde_2020_500_FU_df, Gemeinde_2020_1000_FU_df]:
    df['depth_cat_area_perc_fu'] = (df.FU_area / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_FU_baseline = Gemeinde_2020_5_FU_df[Gemeinde_2020_5_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_10_FU_baseline = Gemeinde_2020_10_FU_df[Gemeinde_2020_10_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_20_FU_baseline = Gemeinde_2020_20_FU_df[Gemeinde_2020_20_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_50_FU_baseline = Gemeinde_2020_50_FU_df[Gemeinde_2020_50_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_100_FU_baseline = Gemeinde_2020_100_FU_df[Gemeinde_2020_100_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_200_FU_baseline = Gemeinde_2020_200_FU_df[Gemeinde_2020_200_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_500_FU_baseline = Gemeinde_2020_500_FU_df[Gemeinde_2020_500_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]
Gemeinde_2020_1000_FU_baseline = Gemeinde_2020_1000_FU_df[Gemeinde_2020_1000_FU_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'FU_area']]

In [ ]:
for y in [5, 10, 20, 50, 100, 200, 500, 1000]:
    df_name = f'Gemeinde_2020_{y}_FU_baseline'
    df = globals()[df_name]
    globals()[df_name] = df.rename(columns={'FU_area': 'FU_area_Jul16'})

In [ ]:
for df_name in ['Gemeinde_2020_5_FU_baseline', 'Gemeinde_2020_10_FU_baseline', 'Gemeinde_2020_20_FU_baseline', 'Gemeinde_2020_50_FU_baseline',
                'Gemeinde_2020_100_FU_baseline', 'Gemeinde_2020_200_FU_baseline', 'Gemeinde_2020_500_FU_baseline', 'Gemeinde_2020_1000_FU_baseline']:
    df = globals()[df_name]
    df = df.rename(columns={'FU_area': 'FU_area_Jul16'})  # Rename first
    df['FU_area_Jul16_non_zero'] = df['FU_area_Jul16'].replace(0, 1e-6)  # Replace zeros
    globals()[df_name] = df

In [ ]:
Gemeinde_2020_5_FU_baseline.head()

In [ ]:
for return_period in [5, 10, 20, 50, 100, 200, 500, 1000]:
    orig_df = globals()[f'Gemeinde_2020_{return_period}_FU_df']
    baseline_df = globals()[f'Gemeinde_2020_{return_period}_FU_baseline']
    merged_df = orig_df.merge(baseline_df, on=['GEM_ID', 'depth_cat'], how='left')
    globals()[f'Gemeinde_2020_{return_period}_FU_df'] = merged_df

In [ ]:
Gemeinde_2020_5_FU_df.head()

In [ ]:
for df in [Gemeinde_2020_5_FU_df, Gemeinde_2020_10_FU_df, Gemeinde_2020_20_FU_df, Gemeinde_2020_50_FU_df,
           Gemeinde_2020_100_FU_df, Gemeinde_2020_200_FU_df, Gemeinde_2020_500_FU_df, Gemeinde_2020_1000_FU_df]:
    df['FU_area_non_zero'] = df.FU_area.replace(0, 1e-6)
    df['pct_growth'] = (df['FU_area_non_zero'] - df['FU_area_Jul16_non_zero']) / df['FU_area_Jul16_non_zero'] * 100

### Create risk level - low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
def risk_level(value):
    if value == '0' or value == '015':
        return 'low_risk'
    else:
        return 'high_risk'

In [ ]:
for df in [Gemeinde_2020_5_FU_df, Gemeinde_2020_10_FU_df, Gemeinde_2020_20_FU_df, Gemeinde_2020_50_FU_df,
           Gemeinde_2020_100_FU_df, Gemeinde_2020_200_FU_df, Gemeinde_2020_500_FU_df, Gemeinde_2020_1000_FU_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Gemeinde_2020_5_FU_df.head()

In [ ]:
Gemeinde_2020_5_jan25_FU = Gemeinde_2020_5_FU_df[Gemeinde_2020_5_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_10_jan25_FU = Gemeinde_2020_10_FU_df[Gemeinde_2020_10_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_20_jan25_FU = Gemeinde_2020_20_FU_df[Gemeinde_2020_20_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_50_jan25_FU = Gemeinde_2020_50_FU_df[Gemeinde_2020_50_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_100_jan25_FU = Gemeinde_2020_100_FU_df[Gemeinde_2020_100_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_200_jan25_FU = Gemeinde_2020_200_FU_df[Gemeinde_2020_200_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_500_jan25_FU = Gemeinde_2020_500_FU_df[Gemeinde_2020_500_FU_df['month_year'] == 'Jan-25']
Gemeinde_2020_1000_jan25_FU = Gemeinde_2020_1000_FU_df[Gemeinde_2020_1000_FU_df['month_year'] == 'Jan-25']

In [ ]:
Gemeinde_2020_5_jan25_FU.head()

In [ ]:
Gemeinde_2020_5_jan25_FU_sa_by_risk = Gemeinde_2020_5_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_10_jan25_FU_sa_by_risk = Gemeinde_2020_10_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_20_jan25_FU_sa_by_risk = Gemeinde_2020_20_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_50_jan25_FU_sa_by_risk = Gemeinde_2020_50_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_100_jan25_FU_sa_by_risk = Gemeinde_2020_100_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_200_jan25_FU_sa_by_risk = Gemeinde_2020_200_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_500_jan25_FU_sa_by_risk = Gemeinde_2020_500_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]
Gemeinde_2020_1000_jan25_FU_sa_by_risk = Gemeinde_2020_1000_jan25_FU.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'FU_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_FU"]

    # Group and sum FU_area_Jul16
    grouped = df.groupby(['GEM_ID', 'risk_level']).sum(numeric_only=True).reset_index()[['GEM_ID', 'risk_level', 'FU_area_Jul16']]

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_FU_sa_by_risk"] = grouped


### Continue to create dataframe separating the flood depth with only two categories: low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
for df in [Gemeinde_2020_5_jan25_FU_sa_by_risk, Gemeinde_2020_10_jan25_FU_sa_by_risk, Gemeinde_2020_20_jan25_FU_sa_by_risk, Gemeinde_2020_50_jan25_FU_sa_by_risk,
           Gemeinde_2020_100_jan25_FU_sa_by_risk, Gemeinde_2020_200_jan25_FU_sa_by_risk, Gemeinde_2020_500_jan25_FU_sa_by_risk, Gemeinde_2020_1000_jan25_FU_sa_by_risk]:
    df.columns = ['GEM_ID', 'risk_level', 'FU_area_by_risk']
    df['FU_area_by_risk_non_zero'] = df.FU_area_by_risk.replace(0, 1e-6)

In [ ]:
Gemeinde_2020_5_jan25_FU_sa_by_risk.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_by_risk']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_by_risk_df'] = merged_df


In [ ]:
Gemeinde_2020_5_jan25_FU_sa_by_risk_df.head()

In [ ]:
for df in [Gemeinde_2020_5_jan25_FU_sa_by_risk_df, Gemeinde_2020_10_jan25_FU_sa_by_risk_df, Gemeinde_2020_20_jan25_FU_sa_by_risk_df, Gemeinde_2020_50_jan25_FU_sa_by_risk_df,
           Gemeinde_2020_100_jan25_FU_sa_by_risk_df, Gemeinde_2020_200_jan25_FU_sa_by_risk_df, Gemeinde_2020_500_jan25_FU_sa_by_risk_df, Gemeinde_2020_1000_jan25_FU_sa_by_risk_df]:

    risk_order = ['low_risk', 'high_risk']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['risk_level_area_perc_fu'] = (df.FU_area_by_risk / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_jan25_FU_sa_by_risk_df.head()

In [ ]:
#Store the processed dataframe as pickle file and read it later
Gemeinde_2020_100_jan25_FU_sa_by_risk_df.to_pickle("Gemeinde_2020_100_jan25_FU_sa_by_risk_df.pkl")

### **Please run the "Create risk level - low risk (<0.15 m) and high risk (>0.15 m)" under Fluvial flooding undefended again** to make this section work properly: Separate risk level further to high (>0.15 m), very high (>0.5 m) and extreme (> 1.5m)

In [ ]:
def very_high_risk(value):
    if value == '150' or value == '150p':
        return 'very_high'
    else:
        return 'other'

In [ ]:
for df in [Gemeinde_2020_5_jan25_FU, Gemeinde_2020_10_jan25_FU, Gemeinde_2020_20_jan25_FU, Gemeinde_2020_50_jan25_FU,
           Gemeinde_2020_100_jan25_FU, Gemeinde_2020_200_jan25_FU, Gemeinde_2020_500_jan25_FU, Gemeinde_2020_1000_jan25_FU]:
    df['very_risky'] = df.depth_cat.map(very_high_risk)

In [ ]:
Gemeinde_2020_5_jan25_FU_sa_very_risky = Gemeinde_2020_5_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_10_jan25_FU_sa_very_risky = Gemeinde_2020_10_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_20_jan25_FU_sa_very_risky = Gemeinde_2020_20_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_50_jan25_FU_sa_very_risky = Gemeinde_2020_50_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_100_jan25_FU_sa_very_risky = Gemeinde_2020_100_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_200_jan25_FU_sa_very_risky = Gemeinde_2020_200_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_500_jan25_FU_sa_very_risky = Gemeinde_2020_500_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]
Gemeinde_2020_1000_jan25_FU_sa_very_risky = Gemeinde_2020_1000_jan25_FU.groupby(['GEM_ID', 'very_risky']).sum('FU_area').reset_index()[['GEM_ID', 'very_risky', 'FU_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_FU"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'very_risky']).sum(numeric_only=True).reset_index()[['GEM_ID', 'very_risky', 'FU_area_Jul16']]
    grouped.columns = ['GEM_ID', 'risk_level', 'FU_area_Jul16']

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_FU_sa_very_risky"] = grouped


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_FU']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'FU_area']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'FU_area']]

    globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_extreme'] = df_150p


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_FU']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'FU_area_Jul16']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'FU_area_Jul16']]

    globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_extreme'] = df_150p


In [ ]:
for df in [Gemeinde_2020_5_jan25_FU_sa_very_risky, Gemeinde_2020_10_jan25_FU_sa_very_risky, Gemeinde_2020_20_jan25_FU_sa_very_risky, Gemeinde_2020_50_jan25_FU_sa_very_risky,
           Gemeinde_2020_100_jan25_FU_sa_very_risky, Gemeinde_2020_200_jan25_FU_sa_very_risky, Gemeinde_2020_500_jan25_FU_sa_very_risky, Gemeinde_2020_1000_jan25_FU_sa_very_risky]:
    df.columns = ['GEM_ID', 'risk_level', 'FU_area']


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_risky_level'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_risky_level'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_FU_sa_risky_level, Gemeinde_2020_10_jan25_FU_sa_risky_level, Gemeinde_2020_20_jan25_FU_sa_risky_level, Gemeinde_2020_50_jan25_FU_sa_risky_level, \
           Gemeinde_2020_100_jan25_FU_sa_risky_level, Gemeinde_2020_200_jan25_FU_sa_risky_level, Gemeinde_2020_500_jan25_FU_sa_risky_level, Gemeinde_2020_1000_jan25_FU_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'FU_area_by_level']
    df['FU_area_by_level_non_zero'] = df.FU_area_by_level.replace(0, 1e-6)

In [ ]:
for df in [Gemeinde_2020_5_Jul16_FU_sa_risky_level, Gemeinde_2020_10_Jul16_FU_sa_risky_level, Gemeinde_2020_20_Jul16_FU_sa_risky_level, Gemeinde_2020_50_Jul16_FU_sa_risky_level, \
           Gemeinde_2020_100_Jul16_FU_sa_risky_level, Gemeinde_2020_200_Jul16_FU_sa_risky_level, Gemeinde_2020_500_Jul16_FU_sa_risky_level, Gemeinde_2020_1000_Jul16_FU_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'FU_area_Jul16_by_level']
    df['FU_area_Jul16_by_level_non_zero'] = df.FU_area_Jul16_by_level.replace(0, 1e-6)

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_FU_sa_risky_level_df'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jul-16'][['GEM_ID', 'Settlement_Area']]
    settle_df.columns = ['GEM_ID', 'Settlement_Area_Jul16']

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_Jul16_FU_sa_risky_level_df'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_FU_sa_risky_level_df, Gemeinde_2020_10_jan25_FU_sa_risky_level_df, Gemeinde_2020_20_jan25_FU_sa_risky_level_df, Gemeinde_2020_50_jan25_FU_sa_risky_level_df,
           Gemeinde_2020_100_jan25_FU_sa_risky_level_df, Gemeinde_2020_200_jan25_FU_sa_risky_level_df, Gemeinde_2020_500_jan25_FU_sa_risky_level_df, Gemeinde_2020_1000_jan25_FU_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['FU_risk_level_area_perc'] = (df.FU_area_by_level / df.Settlement_Area) * 100

In [ ]:
for df in [Gemeinde_2020_5_Jul16_FU_sa_risky_level_df, Gemeinde_2020_10_Jul16_FU_sa_risky_level_df, Gemeinde_2020_20_Jul16_FU_sa_risky_level_df, Gemeinde_2020_50_Jul16_FU_sa_risky_level_df,
           Gemeinde_2020_100_Jul16_FU_sa_risky_level_df, Gemeinde_2020_200_Jul16_FU_sa_risky_level_df, Gemeinde_2020_500_Jul16_FU_sa_risky_level_df, Gemeinde_2020_1000_Jul16_FU_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['FU_risk_level_area_perc_Jul16'] = (df.FU_area_Jul16_by_level / df.Settlement_Area_Jul16) * 100

In [ ]:
Gemeinde_2020_100_jan25_FU_sa_risky_level_df.to_pickle("Gemeinde_2020_100_jan25_FU_sa_risky_level_df.pkl")

In [ ]:
Gemeinde_2020_100_Jul16_FU_sa_risky_level_df.to_pickle("Gemeinde_2020_100_Jul16_FU_sa_risky_level_df.pkl")

## Pluvial Flooding Defended

### data preparation

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
pd_columns = new_cols[:7] + [col for col in new_cols if col.startswith('SA_PD')]

years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_PD'] = globals()[f'Gemeinde_2020_{y}_raw'][pd_columns]


In [ ]:
years = [5, 10, 20, 50, 100, 200, 500, 1000]

for y in years:
    globals()[f'Gemeinde_2020_{y}_PD_long'] = globals()[f'Gemeinde_2020_{y}_PD'].melt(
        id_vars=new_cols[:7], var_name='month_year_depth', value_name='PD_area'
    )

In [ ]:
Gemeinde_2020_5_PD_long.head()

In [ ]:
for df in [Gemeinde_2020_5_PD_long, Gemeinde_2020_10_PD_long, Gemeinde_2020_20_PD_long, Gemeinde_2020_50_PD_long,
           Gemeinde_2020_100_PD_long, Gemeinde_2020_200_PD_long, Gemeinde_2020_500_PD_long, Gemeinde_2020_1000_PD_long]:

    # Extract depth category (e.g., '0', '015', '050', '150', '150p') and month-year (e.g., 'Jul-16')
    df[['depth_cat', 'month_year']] = df['month_year_depth'].str.extract(r'_([0-9]{1,4}p?)_([A-Za-z]{3}-\d{2})$')

In [ ]:
Gemeinde_2020_5_PD_long.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    long_df = globals()[f'Gemeinde_2020_{y}_PD_long']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][['GEM_ID', 'month_year', 'Settlement_Area']]

    merged_df = pd.merge(long_df, settle_df, on=['GEM_ID', 'month_year'], how='left')

    globals()[f'Gemeinde_2020_{y}_PD_df'] = merged_df


In [ ]:
Gemeinde_2020_5_PD_df.head()

In [ ]:
for df in [Gemeinde_2020_5_PD_df, Gemeinde_2020_10_PD_df, Gemeinde_2020_20_PD_df, Gemeinde_2020_50_PD_df,
           Gemeinde_2020_100_PD_df, Gemeinde_2020_200_PD_df, Gemeinde_2020_500_PD_df, Gemeinde_2020_1000_PD_df]:
    df['depth_cat_area_perc_pd'] = (df.PD_area / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_PD_baseline = Gemeinde_2020_5_PD_df[Gemeinde_2020_5_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_10_PD_baseline = Gemeinde_2020_10_PD_df[Gemeinde_2020_10_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_20_PD_baseline = Gemeinde_2020_20_PD_df[Gemeinde_2020_20_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_50_PD_baseline = Gemeinde_2020_50_PD_df[Gemeinde_2020_50_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_100_PD_baseline = Gemeinde_2020_100_PD_df[Gemeinde_2020_100_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_200_PD_baseline = Gemeinde_2020_200_PD_df[Gemeinde_2020_200_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_500_PD_baseline = Gemeinde_2020_500_PD_df[Gemeinde_2020_500_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]
Gemeinde_2020_1000_PD_baseline = Gemeinde_2020_1000_PD_df[Gemeinde_2020_1000_PD_df['month_year'] == 'Jul-16'][['GEM_ID', 'depth_cat', 'PD_area']]

In [ ]:
for y in [5, 10, 20, 50, 100, 200, 500, 1000]:
    df_name = f'Gemeinde_2020_{y}_PD_baseline'
    df = globals()[df_name]
    globals()[df_name] = df.rename(columns={'PD_area': 'PD_area_Jul16'})

In [ ]:
for df_name in ['Gemeinde_2020_5_PD_baseline', 'Gemeinde_2020_10_PD_baseline', 'Gemeinde_2020_20_PD_baseline', 'Gemeinde_2020_50_PD_baseline',
                'Gemeinde_2020_100_PD_baseline', 'Gemeinde_2020_200_PD_baseline', 'Gemeinde_2020_500_PD_baseline', 'Gemeinde_2020_1000_PD_baseline']:
    df = globals()[df_name]
    df = df.rename(columns={'PD_area': 'PD_area_Jul16'})  # Rename first
    df['PD_area_Jul16_non_zero'] = df['PD_area_Jul16'].replace(0, 1e-6)  # Replace zeros
    globals()[df_name] = df

In [ ]:
Gemeinde_2020_5_PD_baseline.head()

In [ ]:
for return_period in [5, 10, 20, 50, 100, 200, 500, 1000]:
    orig_df = globals()[f'Gemeinde_2020_{return_period}_PD_df']
    baseline_df = globals()[f'Gemeinde_2020_{return_period}_PD_baseline']
    merged_df = orig_df.merge(baseline_df, on=['GEM_ID', 'depth_cat'], how='left')
    globals()[f'Gemeinde_2020_{return_period}_PD_df'] = merged_df

In [ ]:
Gemeinde_2020_5_PD_df.head()

In [ ]:
for df in [Gemeinde_2020_5_PD_df, Gemeinde_2020_10_PD_df, Gemeinde_2020_20_PD_df, Gemeinde_2020_50_PD_df,
           Gemeinde_2020_100_PD_df, Gemeinde_2020_200_PD_df, Gemeinde_2020_500_PD_df, Gemeinde_2020_1000_PD_df]:
    df['PD_area_non_zero'] = df.PD_area.replace(0, 1e-6)
    df['pct_growth'] = (df['PD_area_non_zero'] - df['PD_area_Jul16_non_zero']) / df['PD_area_Jul16_non_zero'] * 100

### Create risk level - low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
def risk_level(value):
    if value == '0' or value == '015':
        return 'low_risk'
    else:
        return 'high_risk'

In [ ]:
for df in [Gemeinde_2020_5_PD_df, Gemeinde_2020_10_PD_df, Gemeinde_2020_20_PD_df, Gemeinde_2020_50_PD_df,
           Gemeinde_2020_100_PD_df, Gemeinde_2020_200_PD_df, Gemeinde_2020_500_PD_df, Gemeinde_2020_1000_PD_df]:
    df['risk_level'] = df.depth_cat.map(risk_level)

In [ ]:
Gemeinde_2020_5_PD_df.head()

In [ ]:
Gemeinde_2020_5_jan25_PD = Gemeinde_2020_5_PD_df[Gemeinde_2020_5_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_10_jan25_PD = Gemeinde_2020_10_PD_df[Gemeinde_2020_10_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_20_jan25_PD = Gemeinde_2020_20_PD_df[Gemeinde_2020_20_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_50_jan25_PD = Gemeinde_2020_50_PD_df[Gemeinde_2020_50_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_100_jan25_PD = Gemeinde_2020_100_PD_df[Gemeinde_2020_100_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_200_jan25_PD = Gemeinde_2020_200_PD_df[Gemeinde_2020_200_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_500_jan25_PD = Gemeinde_2020_500_PD_df[Gemeinde_2020_500_PD_df['month_year'] == 'Jan-25']
Gemeinde_2020_1000_jan25_PD = Gemeinde_2020_1000_PD_df[Gemeinde_2020_1000_PD_df['month_year'] == 'Jan-25']

In [ ]:
Gemeinde_2020_5_jan25_PD.head()

In [ ]:
Gemeinde_2020_5_jan25_PD_sa_by_risk = Gemeinde_2020_5_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_10_jan25_PD_sa_by_risk = Gemeinde_2020_10_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_20_jan25_PD_sa_by_risk = Gemeinde_2020_20_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_50_jan25_PD_sa_by_risk = Gemeinde_2020_50_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_100_jan25_PD_sa_by_risk = Gemeinde_2020_100_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_200_jan25_PD_sa_by_risk = Gemeinde_2020_200_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_500_jan25_PD_sa_by_risk = Gemeinde_2020_500_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]
Gemeinde_2020_1000_jan25_PD_sa_by_risk = Gemeinde_2020_1000_jan25_PD.groupby(['GEM_ID', 'risk_level']).sum('FU_area').reset_index()[['GEM_ID', 'risk_level', 'PD_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_PD"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'risk_level']).sum(numeric_only=True).reset_index()[['GEM_ID', 'risk_level', 'PD_area_Jul16']]

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_PD_sa_by_risk"] = grouped


### Continue to create dataframe separating the flood depth with only two categories: low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
for df in [Gemeinde_2020_5_jan25_PD_sa_by_risk, Gemeinde_2020_10_jan25_PD_sa_by_risk, Gemeinde_2020_20_jan25_PD_sa_by_risk, Gemeinde_2020_50_jan25_PD_sa_by_risk,
           Gemeinde_2020_100_jan25_PD_sa_by_risk, Gemeinde_2020_200_jan25_PD_sa_by_risk, Gemeinde_2020_500_jan25_PD_sa_by_risk, Gemeinde_2020_1000_jan25_PD_sa_by_risk]:
    df.columns = ['GEM_ID', 'risk_level', 'PD_area_by_risk']
    df['PD_area_by_risk_non_zero'] = df.PD_area_by_risk.replace(0, 1e-6)

In [ ]:
Gemeinde_2020_5_jan25_PD_sa_by_risk.head()

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_by_risk']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_by_risk_df'] = merged_df


In [ ]:
Gemeinde_2020_5_jan25_PD_sa_by_risk_df.head()

In [ ]:
for df in [Gemeinde_2020_5_jan25_PD_sa_by_risk_df, Gemeinde_2020_10_jan25_PD_sa_by_risk_df, Gemeinde_2020_20_jan25_PD_sa_by_risk_df, Gemeinde_2020_50_jan25_PD_sa_by_risk_df,
           Gemeinde_2020_100_jan25_PD_sa_by_risk_df, Gemeinde_2020_200_jan25_PD_sa_by_risk_df, Gemeinde_2020_500_jan25_PD_sa_by_risk_df, Gemeinde_2020_1000_jan25_PD_sa_by_risk_df]:

    risk_order = ['low_risk', 'high_risk']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['risk_level_area_perc_pd'] = (df.PD_area_by_risk / df.Settlement_Area) * 100

In [ ]:
Gemeinde_2020_5_jan25_PD_sa_by_risk_df.head()

In [ ]:
#Store the processed dataframe to pickle file and read it later
Gemeinde_2020_100_jan25_PD_sa_by_risk_df.to_pickle('Gemeinde_2020_100_jan25_PD_sa_by_risk_df.pkl')

### **Please run the "Create risk level - low risk (<0.15 m) and high risk (>0.15 m)" under Pluvial flooding undefended again** to make this section work properly: Separate risk level further to high (>0.15 m), very high (>0.5 m) and extreme (> 1.5m)

In [ ]:
def very_high_risk(value):
    if value == '150' or value == '150p':
        return 'very_high'
    else:
        return 'other'

In [ ]:
for df in [Gemeinde_2020_5_jan25_PD, Gemeinde_2020_10_jan25_PD, Gemeinde_2020_20_jan25_PD, Gemeinde_2020_50_jan25_PD,
           Gemeinde_2020_100_jan25_PD, Gemeinde_2020_200_jan25_PD, Gemeinde_2020_500_jan25_PD, Gemeinde_2020_1000_jan25_PD]:
    df['very_risky'] = df.depth_cat.map(very_high_risk)

In [ ]:
Gemeinde_2020_5_jan25_PD_sa_very_risky = Gemeinde_2020_5_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_10_jan25_PD_sa_very_risky = Gemeinde_2020_10_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_20_jan25_PD_sa_very_risky = Gemeinde_2020_20_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_50_jan25_PD_sa_very_risky = Gemeinde_2020_50_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_100_jan25_PD_sa_very_risky = Gemeinde_2020_100_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_200_jan25_PD_sa_very_risky = Gemeinde_2020_200_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_500_jan25_PD_sa_very_risky = Gemeinde_2020_500_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]
Gemeinde_2020_1000_jan25_PD_sa_very_risky = Gemeinde_2020_1000_jan25_PD.groupby(['GEM_ID', 'very_risky']).sum('PD_area').reset_index()[['GEM_ID', 'very_risky', 'PD_area']]

In [ ]:
# Define return periods
return_periods = [5, 10, 20, 50, 100, 200, 500, 1000]

# Loop and create variables dynamically
for rp in return_periods:
    # Access the correct dataframe (e.g., Gemeinde_2020_5_jan25_MAX)
    df = globals()[f"Gemeinde_2020_{rp}_jan25_PD"]

    # Group and sum MAX_area_Jul16
    grouped = df.groupby(['GEM_ID', 'very_risky']).sum(numeric_only=True).reset_index()[['GEM_ID', 'very_risky', 'PD_area_Jul16']]
    grouped.columns = ['GEM_ID', 'risk_level', 'PD_area_Jul16']

    # Assign to a new variable with the desired name
    globals()[f"Gemeinde_2020_{rp}_Jul16_PD_sa_very_risky"] = grouped


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_PD']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'PD_area']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'PD_area']]

    globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_extreme'] = df_150p


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df = globals()[f'Gemeinde_2020_{y}_jan25_PD']

    df_150p = df[df['depth_cat'] == '150p'][['GEM_ID', 'depth_cat', 'PD_area_Jul16']].copy()
    df_150p['risk_level'] = 'extreme'
    df_150p =  df_150p[['GEM_ID', 'risk_level', 'PD_area_Jul16']]

    globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_extreme'] = df_150p


In [ ]:
for df in [Gemeinde_2020_5_jan25_PD_sa_very_risky, Gemeinde_2020_10_jan25_PD_sa_very_risky, Gemeinde_2020_20_jan25_PD_sa_very_risky, Gemeinde_2020_50_jan25_PD_sa_very_risky,
           Gemeinde_2020_100_jan25_PD_sa_very_risky, Gemeinde_2020_200_jan25_PD_sa_very_risky, Gemeinde_2020_500_jan25_PD_sa_very_risky, Gemeinde_2020_1000_jan25_PD_sa_very_risky]:
    df.columns = ['GEM_ID', 'risk_level', 'PD_area']


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_risky_level'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    df_risk = globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_by_risk']
    df_very_risk = globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_very_risky']
    df_extreme = globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_extreme']
    merged_df = pd.concat([df_risk[df_risk.risk_level == 'high_risk'], df_very_risk[df_very_risk.risk_level == 'very_high'], df_extreme], ignore_index=True)

    globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_risky_level'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_PD_sa_risky_level, Gemeinde_2020_10_jan25_PD_sa_risky_level, Gemeinde_2020_20_jan25_PD_sa_risky_level, Gemeinde_2020_50_jan25_PD_sa_risky_level, \
           Gemeinde_2020_100_jan25_PD_sa_risky_level, Gemeinde_2020_200_jan25_PD_sa_risky_level, Gemeinde_2020_500_jan25_PD_sa_risky_level, Gemeinde_2020_1000_jan25_PD_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'PD_area_by_level']
    df['PD_area_by_level_non_zero'] = df.PD_area_by_level.replace(0, 1e-6)

In [ ]:
for df in [Gemeinde_2020_5_Jul16_PD_sa_risky_level, Gemeinde_2020_10_Jul16_PD_sa_risky_level, Gemeinde_2020_20_Jul16_PD_sa_risky_level, Gemeinde_2020_50_Jul16_PD_sa_risky_level, \
           Gemeinde_2020_100_Jul16_PD_sa_risky_level, Gemeinde_2020_200_Jul16_PD_sa_risky_level, Gemeinde_2020_500_Jul16_PD_sa_risky_level, Gemeinde_2020_1000_Jul16_PD_sa_risky_level]:
    df.columns = ['GEM_ID', 'risk_level', 'PD_area_Jul16_by_level']
    df['PD_area_Jul16_by_level_non_zero'] = df.PD_area_Jul16_by_level.replace(0, 1e-6)

In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jan-25'][['GEM_ID', 'Settlement_Area']]

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_jan25_PD_sa_risky_level_df'] = merged_df


In [ ]:
years = ['5', '10', '20', '50', '100', '200', '500', '1000']

for y in years:
    sa_by_risk_df = globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_risky_level']
    settle_df = globals()[f'Gemeinde_2020_{y}_SA_long'][globals()[f'Gemeinde_2020_{y}_SA_long'].month_year=='Jul-16'][['GEM_ID', 'Settlement_Area']]
    settle_df.columns = ['GEM_ID', 'Settlement_Area_Jul16']

    merged_df = pd.merge(sa_by_risk_df, settle_df, on='GEM_ID', how='left')

    globals()[f'Gemeinde_2020_{y}_Jul16_PD_sa_risky_level_df'] = merged_df


In [ ]:
for df in [Gemeinde_2020_5_jan25_PD_sa_risky_level_df, Gemeinde_2020_10_jan25_PD_sa_risky_level_df, Gemeinde_2020_20_jan25_PD_sa_risky_level_df, Gemeinde_2020_50_jan25_PD_sa_risky_level_df,
           Gemeinde_2020_100_jan25_PD_sa_risky_level_df, Gemeinde_2020_200_jan25_PD_sa_risky_level_df, Gemeinde_2020_500_jan25_PD_sa_risky_level_df, Gemeinde_2020_1000_jan25_PD_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['PD_risk_level_area_perc'] = (df.PD_area_by_level / df.Settlement_Area) * 100

In [ ]:
for df in [Gemeinde_2020_5_Jul16_PD_sa_risky_level_df, Gemeinde_2020_10_Jul16_PD_sa_risky_level_df, Gemeinde_2020_20_Jul16_PD_sa_risky_level_df, Gemeinde_2020_50_Jul16_PD_sa_risky_level_df,
           Gemeinde_2020_100_Jul16_PD_sa_risky_level_df, Gemeinde_2020_200_Jul16_PD_sa_risky_level_df, Gemeinde_2020_500_Jul16_PD_sa_risky_level_df, Gemeinde_2020_1000_Jul16_PD_sa_risky_level_df]:

    risk_order = ['high_risk', 'very_high', 'extreme']
    df['risk_level'] = pd.Categorical(df['risk_level'], categories=risk_order, ordered=True)

    df['PD_risk_level_area_perc_Jul16'] = (df.PD_area_Jul16_by_level / df.Settlement_Area_Jul16) * 100

In [ ]:
Gemeinde_2020_100_jan25_PD_sa_risky_level_df.to_pickle("Gemeinde_2020_100_jan25_PD_sa_risky_level_df.pkl")

In [ ]:
Gemeinde_2020_100_Jul16_PD_sa_risky_level_df.to_pickle("Gemeinde_2020_100_Jul16_PD_sa_risky_level_df.pkl")

# Add protection data - dikes and dunes

## Dikes

### Read in preprocessed dataframe: risk level - low risk (<0.15 m) and high risk (>0.15 m)

In [ ]:
# Base path to the pickle files
base_path = ''

# File suffixes to load
suffixes = ['MAX', 'CU', 'FU', 'PD']

# Common prefix and postfix for all file names
prefix = 'Gemeinde_2020_100_jan25_'
postfix = '_sa_by_risk_df.pkl'

# Loop through suffixes and load each pickle file
for suffix in suffixes:
    filename = f'{prefix}{suffix}{postfix}'
    full_path = os.path.join(base_path, filename)

    with open(full_path, 'rb') as f:
        # Assign to a variable with the name of the file (without extension)
        var_name = filename.replace('.pkl', '')
        globals()[var_name] = pickle.load(f)


In [ ]:
Gemeinde_2020_100_jan25_MAX_sa_high_risk_df = Gemeinde_2020_100_jan25_MAX_sa_by_risk_df[Gemeinde_2020_100_jan25_MAX_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_100_jan25_CU_sa_high_risk_df = Gemeinde_2020_100_jan25_CU_sa_by_risk_df[Gemeinde_2020_100_jan25_CU_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_100_jan25_FU_sa_high_risk_df = Gemeinde_2020_100_jan25_FU_sa_by_risk_df[Gemeinde_2020_100_jan25_FU_sa_by_risk_df.risk_level == 'high_risk']
Gemeinde_2020_100_jan25_PD_sa_high_risk_df = Gemeinde_2020_100_jan25_PD_sa_by_risk_df[Gemeinde_2020_100_jan25_PD_sa_by_risk_df.risk_level == 'high_risk']

In [ ]:
Gemeinde_2020_100_jan25_sa_high_risk_df_all = \
Gemeinde_2020_100_jan25_MAX_sa_high_risk_df.merge(Gemeinde_2020_100_jan25_CU_sa_high_risk_df[['GEM_ID', 'CU_area_by_risk', 'risk_level_area_perc_cu']], on='GEM_ID') \
                                           .merge(Gemeinde_2020_100_jan25_FU_sa_high_risk_df[['GEM_ID', 'FU_area_by_risk', 'risk_level_area_perc_fu']], on='GEM_ID') \
                                           .merge(Gemeinde_2020_100_jan25_PD_sa_high_risk_df[['GEM_ID', 'PD_area_by_risk', 'risk_level_area_perc_pd']], on='GEM_ID')

### Read in preprocessed dataframe: Separate risk level further to high (>0.15 m), very high (>0.5 m) and extreme (> 1.5m)

In [ ]:
# Jan-25 dataset
# Base path to the pickle files
base_path = ''

# File suffixes to load
suffixes = ['MAX', 'CU', 'FU', 'PD']

# Common prefix and postfix for all file names
prefix = 'Gemeinde_2020_100_jan25_'
postfix = '_sa_risky_level_df.pkl'

# Loop through suffixes and load each pickle file
for suffix in suffixes:
    filename = f'{prefix}{suffix}{postfix}'
    full_path = os.path.join(base_path, filename)

    with open(full_path, 'rb') as f:
        # Assign to a variable with the name of the file (without extension)
        var_name = filename.replace('.pkl', '')
        globals()[var_name] = pickle.load(f)


In [ ]:
# Jul-16 dataset as the baseline
# Base path to the pickle files
base_path = ''

# File suffixes to load
suffixes = ['MAX', 'CU', 'FU', 'PD']

# Common prefix and postfix for all file names
prefix = 'Gemeinde_2020_100_Jul16_'
postfix = '_sa_risky_level_df.pkl'

# Loop through suffixes and load each pickle file
for suffix in suffixes:
    filename = f'{prefix}{suffix}{postfix}'
    full_path = os.path.join(base_path, filename)

    with open(full_path, 'rb') as f:
        # Assign to a variable with the name of the file (without extension)
        var_name = filename.replace('.pkl', '')
        globals()[var_name] = pickle.load(f)


In [ ]:
Gemeinde_2020_100_jan25_MAX_above_050_df = Gemeinde_2020_100_jan25_MAX_sa_risky_level_df[Gemeinde_2020_100_jan25_MAX_sa_risky_level_df.risk_level == 'very_high']
Gemeinde_2020_100_jan25_CU_above_050_df = Gemeinde_2020_100_jan25_CU_sa_risky_level_df[Gemeinde_2020_100_jan25_CU_sa_risky_level_df.risk_level == 'very_high']
Gemeinde_2020_100_jan25_FU_above_050_df = Gemeinde_2020_100_jan25_FU_sa_risky_level_df[Gemeinde_2020_100_jan25_FU_sa_risky_level_df.risk_level == 'very_high']
Gemeinde_2020_100_jan25_PD_above_050_df = Gemeinde_2020_100_jan25_PD_sa_risky_level_df[Gemeinde_2020_100_jan25_PD_sa_risky_level_df.risk_level == 'very_high']

In [ ]:
# Define flood types
flood_types = ["MAX", "CU", "FU", "PD"]

# Loop over each flood type and filter for 'very_high'
for flood_type in flood_types:
    # Build the source DataFrame name
    df_name = f"Gemeinde_2020_100_Jul16_{flood_type}_sa_risky_level_df"

    # Access the DataFrame and filter
    filtered_df = globals()[df_name][globals()[df_name].risk_level == 'very_high']

    # Save the filtered DataFrame under a new variable name
    output_name = f"Gemeinde_2020_100_Jul16_{flood_type}_above_050_df"
    globals()[output_name] = filtered_df

In [ ]:
# Define flood types
flood_types = ["MAX", "CU", "FU", "PD"]

# Loop over each flood type and filter for 'very_high'
for flood_type in flood_types:
    # Build the source DataFrame name
    df_name = f"Gemeinde_2020_100_Jul16_{flood_type}_sa_risky_level_df"

    # Access the DataFrame and filter
    filtered_df = globals()[df_name][globals()[df_name].risk_level == 'high_risk']

    # Save the filtered DataFrame under a new variable name
    output_name = f"Gemeinde_2020_100_Jul16_{flood_type}_sa_high_risk_df"
    globals()[output_name] = filtered_df

In [ ]:
Gemeinde_2020_100_jan25_above_050_df_all = \
Gemeinde_2020_100_jan25_MAX_above_050_df.merge(Gemeinde_2020_100_jan25_CU_above_050_df[['GEM_ID', 'CU_area_by_level', 'CU_risk_level_area_perc']], on='GEM_ID') \
                                        .merge(Gemeinde_2020_100_jan25_FU_above_050_df[['GEM_ID', 'FU_area_by_level', 'FU_risk_level_area_perc']], on='GEM_ID') \
                                        .merge(Gemeinde_2020_100_jan25_PD_above_050_df[['GEM_ID', 'PD_area_by_level', 'PD_risk_level_area_perc']], on='GEM_ID')

In [ ]:
Gemeinde_2020_100_Jul16_above_050_df_all = \
Gemeinde_2020_100_Jul16_MAX_above_050_df.merge(Gemeinde_2020_100_Jul16_CU_above_050_df[['GEM_ID', 'CU_area_Jul16_by_level', 'CU_risk_level_area_perc_Jul16']], on='GEM_ID') \
                                        .merge(Gemeinde_2020_100_Jul16_FU_above_050_df[['GEM_ID', 'FU_area_Jul16_by_level', 'FU_risk_level_area_perc_Jul16']], on='GEM_ID') \
                                        .merge(Gemeinde_2020_100_Jul16_PD_above_050_df[['GEM_ID', 'PD_area_Jul16_by_level', 'PD_risk_level_area_perc_Jul16']], on='GEM_ID')

In [ ]:
Gemeinde_2020_100_Jul16_sa_high_risk_df_all = \
Gemeinde_2020_100_Jul16_MAX_sa_high_risk_df.merge(Gemeinde_2020_100_Jul16_CU_sa_high_risk_df[['GEM_ID', 'CU_area_Jul16_by_level', 'CU_risk_level_area_perc_Jul16']], on='GEM_ID') \
                                           .merge(Gemeinde_2020_100_Jul16_FU_sa_high_risk_df[['GEM_ID', 'FU_area_Jul16_by_level', 'FU_risk_level_area_perc_Jul16']], on='GEM_ID') \
                                           .merge(Gemeinde_2020_100_Jul16_PD_sa_high_risk_df[['GEM_ID', 'PD_area_Jul16_by_level', 'PD_risk_level_area_perc_Jul16']], on='GEM_ID')

### read dike and dune files, continue analysis: **PLEASE update the file path to read in the raw file**

In [ ]:
layers = fiona.listlayers('/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/dike_dune_GIS_file/Hochwasserabwehrinfrastruktur.gdb')
print(layers)

In [ ]:
Gemeinde_2020_100_jan25_sa_high_risk_all_gdf = admin3_boundary.merge(Gemeinde_2020_100_jan25_sa_high_risk_df_all, left_on='OBJID', right_on='GEM_ID', how='right')

In [ ]:
Gemeinde_2020_100_jan25_above_050_all_gdf = admin3_boundary.merge(Gemeinde_2020_100_jan25_above_050_df_all, left_on='OBJID', right_on='GEM_ID', how='right')

In [ ]:
dike = gpd.read_file('/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/dike_dune_GIS_file/Hochwasserabwehrinfrastruktur.gdb', layer='Deiche')

In [ ]:
dune = gpd.read_file('/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/dike_dune_GIS_file/Hochwasserabwehrinfrastruktur.gdb', layer='Schutzdünen')

### dike intersection and other data preprocessing for plotting

In [ ]:
print(Gemeinde_2020_100_jan25_sa_high_risk_all_gdf.crs)
print(dike.crs)

In [ ]:
utm_crs = "EPSG:25832"  # ETRS89 / UTM zone 32N
Gemeinde_2020_100_jan25_sa_high_risk_all_utm = Gemeinde_2020_100_jan25_sa_high_risk_all_gdf.to_crs(utm_crs)
dike_utm = dike.to_crs(utm_crs)

In [ ]:
def get_intersecting_lines(poly):
    clipped = dike_utm[dike_utm.intersects(poly)]
    intersected = clipped.intersection(poly)

    # Collect all line parts (flatten MultiLineStrings)
    lines = []
    for geom in intersected:
        if geom.is_empty:
            continue
        if geom.geom_type == "LineString":
            lines.append(geom)
        elif geom.geom_type == "MultiLineString":
            lines.extend([g for g in geom.geoms if g.geom_type == "LineString"])

    return MultiLineString(lines) if lines else None


In [ ]:
# Apply the function
Gemeinde_2020_100_jan25_sa_high_risk_all_utm["intersecting_dikes"] = \
    Gemeinde_2020_100_jan25_sa_high_risk_all_utm["geometry"].apply(get_intersecting_lines)

In [ ]:
from shapely.geometry import MultiLineString

# Calculate total length of intersecting dikes per row
Gemeinde_2020_100_jan25_sa_high_risk_all_utm["dike_length"] = \
    Gemeinde_2020_100_jan25_sa_high_risk_all_utm["intersecting_dikes"].apply(
        lambda geom: geom.length if geom else 0
    )

In [ ]:
# Save the data to a pickle file and reload it later if the session needs to be restarted due to running out of RAM.
Gemeinde_2020_100_jan25_sa_high_risk_all_utm.to_pickle('Gemeinde_2020_100_jan25_sa_high_risk_all_utm.pkl')

In [ ]:
with open('Gemeinde_2020_100_jan25_sa_high_risk_all_utm.pkl', 'rb') as f:
    Gemeinde_2020_100_jan25_sa_high_risk_all_utm = pickle.load(f)

In [ ]:
# Select the columns you want to keep (e.g., GEN, dike_length, intersecting_dikes as geometry)
intersected_dikes_gdf = gpd.GeoDataFrame(
    Gemeinde_2020_100_jan25_sa_high_risk_all_utm.drop(columns='geometry').copy(),
    geometry="intersecting_dikes",
    crs=Gemeinde_2020_100_jan25_sa_high_risk_all_utm.crs
)


# Drop empty geometries (if any)
intersected_lines_gdf = intersected_dikes_gdf[~intersected_dikes_gdf.geometry.is_empty & intersected_dikes_gdf.geometry.notnull()]

In [ ]:
## Plot to verify that the spatial join was performed correctly; this plot is for checking only, not for publication.
# fig, ax = plt.subplots(figsize=(10, 10))
# Gemeinde_2020_100_jan25_sa_high_risk_all_utm.plot(ax=ax, facecolor='lightgrey', edgecolor='black')
# intersected_lines_gdf.plot(ax=ax, color='red', linewidth=1)
# plt.title("Gemeindes with Intersecting Dike Segments")
# plt.show()

In [ ]:
Gemeinde_2020_100_jan25_sa_high_risk_all_utm = Gemeinde_2020_100_jan25_sa_high_risk_all_utm.merge(
                                                Gemeinde_2020_100_Jul16_sa_high_risk_df_all.drop(columns='risk_level'),
                                                on='GEM_ID',
                                                how='left'
                                               )

In [ ]:
Gemeinde_2020_100_jan25_sa_high_risk_all_utm.columns

In [ ]:
Gemeinde_2020_100_jan25_sa_high_risk_all_utm.rename(columns={'Max_area_by_risk': 'MAX_area_by_risk'}, inplace=True)

In [ ]:
# Define flood types
flood_types = ["MAX", "CU", "FU", "PD"]

# Loop through each flood type and calculate growth
for flood_type in flood_types:
    # Build column names
    col_current = f"{flood_type}_area_by_risk"
    col_base = f"{flood_type}_area_Jul16_by_level"
    col_growth = f"{flood_type.lower()}_pct_growth"  # Lowercase for the new column

    # Calculate % growth and store in a new column
    Gemeinde_2020_100_jan25_sa_high_risk_all_utm[col_growth] = (
        (Gemeinde_2020_100_jan25_sa_high_risk_all_utm[col_current] - Gemeinde_2020_100_jan25_sa_high_risk_all_utm[col_base])
        / (Gemeinde_2020_100_jan25_sa_high_risk_all_utm[col_base] + 1e-6)
    )

In [ ]:
Gemeinde_2020_100_jan25_sa_high_risk_all_utm.head()

In [ ]:
Gemeinde_2020_100_jan25_above_050_all_utm = Gemeinde_2020_100_jan25_sa_high_risk_all_utm\
                                            [['OBJID', 'BEGINN', 'ADE', 'GF', 'BSG', 'ARS', 'AGS', 'SDV_ARS', 'GEN',
                                              'BEZ', 'IBZ', 'BEM', 'NBD', 'SN_L', 'SN_R', 'SN_K', 'SN_V1', 'SN_V2',
                                              'SN_G', 'FK_S3', 'NUTS', 'ARS_0', 'AGS_0', 'WSK', 'DLM_ID', 'geometry',
                                              'GEM_ID',
                                              'intersecting_dikes', 'dike_length']].merge(Gemeinde_2020_100_jan25_above_050_df_all, on='GEM_ID', how='right')

In [ ]:
# Save the data to a pickle file and reload it later if the session needs to be restarted due to running out of RAM.
Gemeinde_2020_100_jan25_above_050_all_utm.to_pickle('Gemeinde_2020_100_jan25_above_050_all_utm.pkl')

In [ ]:
with open('Gemeinde_2020_100_jan25_above_050_all_utm.pkl', 'rb') as f:
    Gemeinde_2020_100_jan25_above_050_all_utm = pickle.load(f)

In [ ]:
Gemeinde_2020_100_jan25_above_050_all_utm = \
Gemeinde_2020_100_jan25_above_050_all_utm.merge(Gemeinde_2020_100_Jul16_above_050_df_all.drop(columns='risk_level'), on='GEM_ID', how='left')

In [ ]:
Gemeinde_2020_100_jan25_MAX_sa_above_050 = Gemeinde_2020_100_jan25_MAX_sa_risky_level_df\
                                          [Gemeinde_2020_100_jan25_MAX_sa_risky_level_df.risk_level == 'very_high']

In [ ]:
Gemeinde_2020_100_jan25_MAX_sa_above_050_utm = Gemeinde_2020_100_jan25_sa_high_risk_all_utm\
                                              [['OBJID', 'BEGINN', 'ADE', 'GF', 'BSG', 'ARS', 'AGS', 'SDV_ARS',
                                                'GEN','BEZ', 'IBZ', 'BEM', 'NBD', 'SN_L', 'SN_R', 'SN_K', 'SN_V1', 'SN_V2',
                                                'SN_G', 'FK_S3', 'NUTS', 'ARS_0', 'AGS_0', 'WSK', 'DLM_ID', 'geometry','GEM_ID', 'dike_length']]\
                                              .merge(Gemeinde_2020_100_jan25_MAX_sa_above_050, on='GEM_ID', how='right')

### dune intersection

In [ ]:
# Ensure same CRS
dune_utm = dune.to_crs(Gemeinde_2020_100_jan25_sa_high_risk_all_utm.crs)

# Drop unwanted column and copy to new GeoDataFrame
Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm = Gemeinde_2020_100_jan25_sa_high_risk_all_utm.drop(columns='intersecting_dikes').copy()

# Initialize an empty list to store intersected geometries
intersected_geoms = []
intersected_areas = []

# Loop over each Gemeinde and compute intersection
for geom in Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.geometry:
    intersection = dune_utm.intersection(geom)
    intersection = intersection[~intersection.is_empty]

    if not intersection.empty:
        merged = intersection.unary_union
        if merged and merged.is_valid:
            intersected_geoms.append(merged)
            intersected_areas.append(merged.area)
        else:
            intersected_geoms.append(None)
            intersected_areas.append(0)
    else:
        intersected_geoms.append(None)
        intersected_areas.append(0)

# Store results
Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm['intersecting_dunes'] = intersected_geoms
Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm['dune_area_sqm'] = intersected_areas


In [ ]:
# Store is as pickle file and read it later
Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.to_pickle('Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.pkl')

In [ ]:
with open('Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.pkl', 'rb') as f:
    Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm = pickle.load(f)

In [ ]:
# Select the columns you want to keep (e.g., GEN, dike_length, intersecting_dikes as geometry)
intersected_dunes_gdf = gpd.GeoDataFrame(
    Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.drop(columns='geometry').copy(),
    geometry="intersecting_dunes",
    crs=Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.crs
)


# Drop empty geometries (if any)
intersected_polydunes_gdf = intersected_dunes_gdf[~intersected_dunes_gdf.geometry.is_empty & intersected_dunes_gdf.geometry.notnull()]

In [ ]:
# # Plot to verify that the spatial join was performed correctly; this plot is for checking only, not for publication.
# gemeinde_gdf= Gemeinde_2020_100_jan25_sa_high_risk_all_dune_utm.copy()

# # Add a flag to Gemeinde for those that intersect with dunes
# gemeinde_gdf['has_dune'] = [geom is not None for geom in intersected_geoms]

# # Subset of Gemeinden with intersecting dunes
# gemeinden_with_dunes = gemeinde_gdf[gemeinde_gdf['has_dune']]

# # Plot
# fig, ax = plt.subplots(figsize=(12, 10))

# # Base: all Gemeinden in light grey
# gemeinde_gdf.plot(ax=ax, facecolor='lightgrey', edgecolor='white', linewidth=0.2)

# # Highlight: Gemeinden with dunes
# gemeinden_with_dunes.plot(ax=ax, facecolor='orange', edgecolor='black', linewidth=0.6, alpha=0.7, label='Gemeinden w/ Dunes')

# # Dunes overlaid: only edges
# dune_utm.boundary.plot(ax=ax, edgecolor='yellow', linewidth=1, label='Dunes')

# # Final touches
# ax.set_title("Gemeinden Intersecting with Dunes")
# ax.axis('off')
# ax.legend(loc='upper left')
# plt.tight_layout()
# plt.show()


### dike density (km per sqkm exposure above 0.5m) with flooding type dominance

In [ ]:
with open('Gemeinde_2020_100_jan25_above_050_all_utm.pkl', 'rb') as f:
    Gemeinde_2020_100_jan25_above_050_all_utm = pickle.load(f)

In [ ]:
Gemeinde_2020_100_jan25_above_050_all_utm.columns

In [ ]:
len(Gemeinde_2020_100_jan25_above_050_all_utm)

In [ ]:
Gemeinde_2020_100_jan25_above_050_all_utm['cu_max_ratio'] = Gemeinde_2020_100_jan25_above_050_all_utm.CU_area_by_level / Gemeinde_2020_100_jan25_above_050_all_utm.MAX_area_by_level_non_zero
Gemeinde_2020_100_jan25_above_050_all_utm['fu_max_ratio'] = Gemeinde_2020_100_jan25_above_050_all_utm.FU_area_by_level / Gemeinde_2020_100_jan25_above_050_all_utm.MAX_area_by_level_non_zero
Gemeinde_2020_100_jan25_above_050_all_utm['pd_max_ratio'] = Gemeinde_2020_100_jan25_above_050_all_utm.PD_area_by_level / Gemeinde_2020_100_jan25_above_050_all_utm.MAX_area_by_level_non_zero

In [ ]:
# Select only the relevant ratio columns
ratio_cols = ['cu_max_ratio', 'fu_max_ratio', 'pd_max_ratio']

# Use idxmax to find the column with the maximum value per row
Gemeinde_2020_100_jan25_above_050_all_utm['dominant_type'] = (
    Gemeinde_2020_100_jan25_above_050_all_utm[ratio_cols]
    .idxmax(axis=1)
    .str.replace('_max_ratio', '_dominant')
)

In [ ]:
exposed_above_050_utm = Gemeinde_2020_100_jan25_above_050_all_utm[Gemeinde_2020_100_jan25_above_050_all_utm.MAX_area_by_level>0].reset_index(drop=True)

In [ ]:
exposed_above_050_utm['dike_km_per_sqkm'] = (exposed_above_050_utm.dike_length / 1000) / exposed_above_050_utm.MAX_area_by_level

### **PLEASE go back to the section Combined(max) flooding then re-run the "data preparation" and the "Create risk level - low risk (<0.15 m) and high risk (>0.15 m)"** (if you lost the variable "Gemeinde_2020_100_jan25_MAX" because of the session re-start)

In [ ]:
#borrow land col from a previous section so I can include the land name into the dataframe for plotting
# PLEASE go to section Combined(max) flooding then re-run the "data preparation" and the "Create risk level - low risk (<0.15 m) and high risk (>0.15 m)" if you lost the variable because of the session re-start
Gemeinde_2020_100_jan25_MAX_p0 = Gemeinde_2020_100_jan25_MAX[Gemeinde_2020_100_jan25_MAX.depth_cat=='0'].copy().reset_index(drop=True)

In [ ]:
Gemeinde_2020_100_jan25_MAX_p0.columns

In [ ]:
exposed_above_050_with_land_utm = exposed_above_050_utm.merge(Gemeinde_2020_100_jan25_MAX_p0[['GEM_ID', 'LAN_NAME']], on='GEM_ID', how='left').reset_index(drop=True)

In [ ]:
# Get the actual hex colors
coastal_color = to_hex(cm.Blues(0.6))
fluvial_color = to_hex(cm.Greens(0.6))
pluvial_color = to_hex(cm.Purples(0.6))

print("Coastal Dominant (Blues 0.6):", coastal_color)
print("Fluvial Dominant (Greens 0.6):", fluvial_color)
print("Pluvial Dominant (Purples 0.6):", pluvial_color)

In [ ]:
exposed_above_050_with_land_utm['lg_dike_km_per_sqkm'] = np.log10(exposed_above_050_with_land_utm.dike_km_per_sqkm)

In [ ]:
# Save the processed dataframe for later use in plotting, in case the notebook must be restarted because of RAM limitations.
exposed_above_050_with_land_utm.to_pickle("exposed_above_050_Jan25_dike_density_strip.pkl")

In [ ]:
with open("exposed_above_050_Jan25_dike_density_strip.pkl", 'rb') as f:
    exposed_above_050_with_land_utm = pickle.load(f)

In [ ]:
# Subset to coastal and fluvial dominant only
# Create a new column combining small states
protected_exposed_above_050_cu_fu_utm = exposed_above_050_with_land_utm[(exposed_above_050_with_land_utm.dike_length > 0) & (exposed_above_050_with_land_utm.dominant_type.isin(['cu_dominant', 'fu_dominant']))]
protected_exposed_above_050_cu_fu_utm['LAN_NAME_MOD'] = protected_exposed_above_050_cu_fu_utm['LAN_NAME'].replace(
    {'Bremen': 'City states', 'Hamburg': 'City states', 'Berlin': 'City states'}
)

# Compute means
violin_land_order = protected_exposed_above_050_cu_fu_utm.groupby('LAN_NAME_MOD')['lg_dike_km_per_sqkm'].mean().sort_values(ascending=False).index.tolist()


In [ ]:
protected_exposed_above_050_cu_fu_utm.groupby('LAN_NAME_MOD')['lg_dike_km_per_sqkm'].mean().sort_values(ascending=False)

In [ ]:
# Save setup
sns.set(style="whitegrid")
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_protection_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)

# Custom colors
coastal_color = to_hex(cm.Blues(0.6))
fluvial_color = to_hex(cm.Greens(0.6))

flood_colors = {
    'cu_dominant': coastal_color,
    'fu_dominant': fluvial_color
}

flood_type_map = {
    'cu_dominant': 'Coastal Dominant',
    'fu_dominant': 'Fluvial Dominant'
}

# Compute std and percentiles
grouped = protected_exposed_above_050_cu_fu_utm.groupby("LAN_NAME_MOD")
std_by_land = grouped["lg_dike_km_per_sqkm"].std().round(2)
p75_by_land = grouped["lg_dike_km_per_sqkm"].quantile(0.75)
p90_by_land = grouped["lg_dike_km_per_sqkm"].quantile(0.90)


# Plot
fig, ax = plt.subplots(figsize=(12, 8))
sns.set(style="whitegrid")

sns.stripplot(
    data=protected_exposed_above_050_cu_fu_utm,
    x="LAN_NAME_MOD",
    y="lg_dike_km_per_sqkm",
    hue="dominant_type",
    palette=flood_colors,
    dodge=True,
    jitter=0.25,
    alpha=0.7,
    ax=ax,
    order=violin_land_order,
)

sns.boxplot(
    data=protected_exposed_above_050_cu_fu_utm,
    x="LAN_NAME_MOD",
    y="lg_dike_km_per_sqkm",
    hue="dominant_type",
    palette=flood_colors,
    dodge=True,
    medianprops={'color': 'k', 'linestyle': '-', 'linewidth': 1},
    boxprops={'edgecolor': 'k', 'linestyle': '-', 'linewidth': 1},
    whiskerprops={'visible': False},
    capprops={'visible': False},
    showbox=False,
    showfliers=False,
    showcaps=False,
    zorder=10,
    ax=ax,
    order=violin_land_order,
)

# Add percentile lines per land
# Define dodge offsets for flood types
for i, land in enumerate(violin_land_order):
    for flood_type in ["cu_dominant", "fu_dominant"]:
        subset = protected_exposed_above_050_cu_fu_utm[
            (protected_exposed_above_050_cu_fu_utm["LAN_NAME_MOD"] == land) &
            (protected_exposed_above_050_cu_fu_utm["dominant_type"] == flood_type)
        ]

        if subset.empty:
            continue  # Skip if no data for this combo

        # Calculate percentiles from subset y-values
        p75 = np.percentile(subset["lg_dike_km_per_sqkm"], 75)
        p90 = np.percentile(subset["lg_dike_km_per_sqkm"], 90)

        # Dodge shift to align with dots
        dodge_shift = {'cu_dominant': 0.2, 'fu_dominant': -0.2}
        x_pos = i + dodge_shift.get(flood_type, 0)

        # Plot 75th percentile (long dash)
        ax.hlines(
            p75,
            x_pos - 0.18, x_pos + 0.18,
            color='k',
            linestyle='--',
            linewidth=1.2,
            label='75th Percentile' if i == 0 and flood_type == 'cu_dominant' else "",
            zorder=10
        )

        # Plot 90th percentile (dotted/dash-dot)
        ax.hlines(
            p90,
            x_pos - 0.18, x_pos + 0.18,
            color='k',
            linestyle=':',
            linewidth=1.2,
            label='90th Percentile' if i == 0 and flood_type == 'cu_dominant' else "",
            zorder=10
        )


# Add vertical dividers between lands
for i in range(1, len(violin_land_order)):
    ax.axvline(x=i - 0.5, color='gray', linestyle=':', linewidth=0.8, alpha=0.6)

# Labels and legend
ax.set_xlabel("Land", fontsize=14)

# Scientific tick labels with superscripts (10⁻³ style)
def sci_formatter(y, _):
    # Convert log10 value to superscript scientific notation
    return f"$10^{{{int(y)}}}$"

ax.yaxis.set_major_formatter(ticker.FuncFormatter(sci_formatter))
ax.set_ylabel("Dike density (km/km$^2$)", fontsize=14)

# Rotate x labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=11)

# Custom legend handles
flood_handles = [
    mpatches.Patch(color=coastal_color, label='Coastal'),
    mpatches.Patch(color=fluvial_color, label='Fluvial')
]

line_50 = mlines.Line2D([], [], color='black', linestyle='-', label='50th')
line_75 = mlines.Line2D([], [], color='black', linestyle='--', label='75th')
line_90 = mlines.Line2D([], [], color='black', linestyle=':', label='90th')

# First legend (Percentiles)
legend1 = ax.legend(
    handles=[line_50, line_75, line_90],
    title="Percentiles",
    fontsize=12,
    title_fontsize=14,
    loc='upper left',
    bbox_to_anchor=(-0.15, -0.15)  # bottom left, slightly outside
)

# Second legend (Flood Type) stacked below
legend2 = ax.legend(
    handles=flood_handles,
    title="Flood type",
    fontsize=12,
    title_fontsize=14,
    loc='upper left',
    bbox_to_anchor=(-0.15, -0.55)  # further down
)

# Add first legend back
ax.add_artist(legend1)

plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
# === SAVE FIGURE ===
file_base = "exposed_above_050_Jan25_dike_density_dist_sorted_cu_fu_strip"
# png_path = os.path.join(save_path, file_base + ".png")
svg_path = os.path.join(save_path, file_base + ".svg")

plt.tight_layout()

# High-resolution PNG
# fig.savefig(png_path, dpi=300, bbox_inches="tight")

# Vector-format SVG (scales perfectly in Illustrator / Inkscape)
# fig.savefig(svg_path, format="svg", bbox_inches="tight")

# print(f"✅ Saved both PNG and SVG to:\n{save_path}")

plt.show()



In [ ]:
# This plot provides information for a table in the appendix; the plot itself is not included in the manuscript.
# Save setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_protection_plots"
os.makedirs(save_path, exist_ok=True)
file_name = "exposed_above_050_Jan25_dike_density_dist_sorted_cu_fu_strip_w_std_by_flood_type.png"
full_path = os.path.join(save_path, file_name)

# Custom colors
coastal_color = to_hex(cm.Blues(0.6))
fluvial_color = to_hex(cm.Greens(0.6))

flood_colors = {
    'cu_dominant': coastal_color,
    'fu_dominant': fluvial_color
}

flood_type_map = {
    'cu_dominant': 'Coastal Dominant',
    'fu_dominant': 'Fluvial Dominant'
}

# Compute std and percentiles
std_by_combo = (
    protected_exposed_above_050_cu_fu_utm
    .groupby(["LAN_NAME_MOD", "dominant_type"])["lg_dike_km_per_sqkm"]
    .std()
    .round(2)
    .unstack(fill_value=np.nan)  # fluvial = 'fu_dominant', coastal = 'cu_dominant'
)

# Update x-tick labels with std values
violin_land_order_with_dual_std = []

for land in violin_land_order:
    fluvial_std = std_by_combo.loc[land, 'fu_dominant'] if 'fu_dominant' in std_by_combo.columns and land in std_by_combo.index else np.nan
    coastal_std = std_by_combo.loc[land, 'cu_dominant'] if 'cu_dominant' in std_by_combo.columns and land in std_by_combo.index else np.nan

    # Format with placeholder if missing
    fluvial_fmt = f"{fluvial_std:.2f}" if not np.isnan(fluvial_std) else "-"
    coastal_fmt = f"{coastal_std:.2f}" if not np.isnan(coastal_std) else "-"

    label = f"{land}\n(fluvial={fluvial_fmt}, coastal={coastal_fmt})"
    violin_land_order_with_dual_std.append(label)

land_label_dual_map = dict(zip(violin_land_order, violin_land_order_with_dual_std))
protected_exposed_above_050_cu_fu_utm["LAN_LABEL_STD"] = protected_exposed_above_050_cu_fu_utm["LAN_NAME_MOD"].map(land_label_dual_map)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
sns.set(style="whitegrid")

sns.stripplot(
    data=protected_exposed_above_050_cu_fu_utm,
    x="LAN_LABEL_STD",
    y="lg_dike_km_per_sqkm",
    hue="dominant_type",
    palette=flood_colors,
    dodge=True,
    jitter=0.25,
    alpha=0.7,
    ax=ax,
    order=violin_land_order_with_dual_std,
)

sns.boxplot(
    data=protected_exposed_above_050_cu_fu_utm,
    x="LAN_LABEL_STD",
    y="lg_dike_km_per_sqkm",
    hue="dominant_type",
    palette=flood_colors,
    dodge=True,
    medianprops={'color': 'k', 'linestyle': '-', 'linewidth': 1},
    boxprops={'edgecolor': 'k', 'linestyle': '-', 'linewidth': 1},
    whiskerprops={'visible': False},
    capprops={'visible': False},
    showbox=False,
    showfliers=False,
    showcaps=False,
    zorder=10,
    ax=ax,
    order=violin_land_order_with_dual_std,
)

# Add percentile lines per land
# Define dodge offsets for flood types
for i, land in enumerate(violin_land_order):
    for flood_type in ["cu_dominant", "fu_dominant"]:
        subset = protected_exposed_above_050_cu_fu_utm[
            (protected_exposed_above_050_cu_fu_utm["LAN_NAME_MOD"] == land) &
            (protected_exposed_above_050_cu_fu_utm["dominant_type"] == flood_type)
        ]

        if subset.empty:
            continue  # Skip if no data for this combo

        # Calculate percentiles from subset y-values
        p75 = np.percentile(subset["lg_dike_km_per_sqkm"], 75)
        p90 = np.percentile(subset["lg_dike_km_per_sqkm"], 90)

        # Dodge shift to align with dots
        dodge_shift = {'cu_dominant': 0.2, 'fu_dominant': -0.2}
        x_pos = i + dodge_shift.get(flood_type, 0)

        # Plot 75th percentile (long dash)
        ax.hlines(
            p75,
            x_pos - 0.18, x_pos + 0.18,
            color='k',
            linestyle='--',
            linewidth=1.2,
            label='75th Percentile' if i == 0 and flood_type == 'cu_dominant' else "",
            zorder=10
        )

        # Plot 90th percentile (dotted/dash-dot)
        ax.hlines(
            p90,
            x_pos - 0.18, x_pos + 0.18,
            color='k',
            linestyle=':',
            linewidth=1.2,
            label='90th Percentile' if i == 0 and flood_type == 'cu_dominant' else "",
            zorder=10
        )


# Add vertical dividers between lands
for i in range(1, len(violin_land_order)):
    ax.axvline(x=i - 0.5, color='gray', linestyle=':', linewidth=0.8, alpha=0.6)

# Labels and legend
ax.set_title('Dike Density by Flood Type and Land (Jan-25)', fontsize=18)
ax.set_xlabel("Land (with Protection Variation by std)", fontsize=14)

def sci_formatter(y, _):
    return f"{10**y:.0e}"

ax.yaxis.set_major_formatter(FuncFormatter(sci_formatter))
ax.set_ylabel("Dike Density (km / sqkm)", fontsize=14)

# Rotate x labels vertically
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=11)

# Custom legend (remove duplicates from dual hue)
flood_handles = [
    mpatches.Patch(color=coastal_color, label='Coastal'),
    mpatches.Patch(color=fluvial_color, label='Fluvial')
]

# Percentile lines legend (lines)
line_50 = mlines.Line2D([], [], color='black', linestyle='-', label='50th')
line_75 = mlines.Line2D([], [], color='black', linestyle='--', label='75th')
line_90 = mlines.Line2D([], [], color='black', linestyle=':', label='90th')

legend1 = ax.legend(
    handles=[line_50, line_75, line_90],
    title="Percentiles",
    fontsize=12,
    title_fontsize=14,
    loc='lower left',
    bbox_to_anchor=(1.02, 0)
)

legend2 = ax.legend(
    handles=flood_handles,
    title="Dominant Flood",
    fontsize=12,
    title_fontsize=14,
    loc='upper left',
    bbox_to_anchor=(1.02, 1)
)

# Add the first legend back so both show up
ax.add_artist(legend1)

plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
gemeinde_plot = Gemeinde_2020_100_jan25_above_050_all_utm.drop(columns='intersecting_dikes').copy().to_crs(epsg=3857)
dune_plot = dune.to_crs(epsg=3857)
dike_plot = dike.to_crs(epsg=3857)

gemeinde_plot['masked_share'] = gemeinde_plot['risk_level_area_perc'].replace(0, np.nan)
gemeinde_plot2 = gemeinde_plot[gemeinde_plot['masked_share'].notna()].copy()

# Step 1: Get natural breaks
# Drop NaNs temporarily for classification
nb = mapclassify.NaturalBreaks(gemeinde_plot2['masked_share'], k=5)

gemeinde_plot2['intensity_bin'] = np.nan
gemeinde_plot2.loc[gemeinde_plot['masked_share'].notna(), 'intensity_bin'] = nb.yb
gemeinde_plot2['intensity_bin'] = gemeinde_plot2['intensity_bin'].astype('Int64')  # optional clean type

# Step 2: Define discrete color ramps per dominant type
cmap_steps = 5
colormaps = {
    'cu_dominant': cm.Blues(np.linspace(0.4, 0.9, cmap_steps)),
    'fu_dominant': cm.Greens(np.linspace(0.4, 0.9, cmap_steps)),
    'pd_dominant': cm.Purples(np.linspace(0.4, 0.9, cmap_steps)),
}

# Step 3: Apply colors based on bin + dominant type
def get_color_natural(row):
    bin_idx = row['intensity_bin']

    if pd.isna(bin_idx):
        return "#d3d3d3"  # Light grey for NaNs

    try:
        bin_idx = int(bin_idx)  # Ensure it's an integer
        cmap = colormaps.get(row['dominant_type'], cm.Greys(np.linspace(0.4, 0.9, cmap_steps)))
        return to_hex(cmap[bin_idx])
    except Exception as e:
        print(f"Error on row {row.name}: {e}")
        return "#000000"  # fallback: black for error

# Then apply
gemeinde_plot2['color'] = gemeinde_plot2.apply(get_color_natural, axis=1)

# Plot
fig, ax = plt.subplots(figsize=(14, 12))
plot = gemeinde_plot2.plot(
    ax=ax,
    color=gemeinde_plot2['color'],
    edgecolor=None,
    linewidth=0.5
)

# 2. Overlay geometry boundaries with NO fill, just borders
gemeinde_plot.boundary.plot(
    ax=ax,
    color='grey',
    linewidth=0.05
)

# Plot all dikes as a single black line
dike_plot.plot(
    ax=ax,
    color='black',
    linewidth=0.5,
    label='Dike'
)

# 2. Overlay geometry boundaries with NO fill, just borders
gemeinde_plot.boundary.plot(
    ax=ax,
    color='grey',
    linewidth=0.05
)

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gemeinde_plot.crs)
# Title and cleanup
# ax.set_title(f"Above 0.5 m Flood Depth Exposure and Dike Protection — 100-Year Return")
ax.axis('off')

# Add north arrow and scale bar
add_north_arrow(ax, scale=0.5, xlim_pos=0.925, ylim_pos=0.8, color='black', text_scaler=4, text_yT=-1.25)
scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
ax.add_artist(scale1)

# === Exposure Intensity: Circle Legend ===
flood_type_colors = {
    'cu_dominant': cm.Blues(np.linspace(0.4, 0.9, len(nb.bins))),
    'fu_dominant': cm.Greens(np.linspace(0.4, 0.9, len(nb.bins))),
    'pd_dominant': cm.Purples(np.linspace(0.4, 0.9, len(nb.bins)))
}

# Choose one representative flood type for color ramp (e.g., 'pd_dominant')
sample_type = 'cu_dominant'  # or 'cu_dominant', 'fu_dominant', etc.

circles = []
for i, b in enumerate(nb.bins):
    lower = int(nb.bins[i - 1]) if i > 0 else int(min(gemeinde_plot['risk_level_area_perc']))
    upper = int(b)
    label = f"{lower}–{upper}%" if i < len(nb.bins) - 1 else f"{lower}+%"

    color = flood_type_colors[sample_type][i]

    circle_marker = Line2D(
        [0], [0],
        marker='o',
        color='None',  # edge color of the marker line (white or none)
        markerfacecolor=color,
        markeredgecolor='None',
        markersize=10,
        label=label
    )
    circles.append(circle_marker)

# === Add Legends ===

# 1. Flood intensity
legend1 = ax.legend(handles=circles,
          title="Exposed \nsettlement",
          loc='upper left',
          fontsize=12,
          title_fontsize=14)
ax.add_artist(legend1)

# 2. Flood type (hue only, optional)
fldtype_handles = [
    mpatches.Patch(color=to_hex(cm.Blues(0.6)), label='Coastal dominant'),
    mpatches.Patch(color=to_hex(cm.Greens(0.6)), label='Fluvial dominant'),
    mpatches.Patch(color=to_hex(cm.Purples(0.6)), label='Pluvial dominant')
]

legend2 = ax.legend(
    handles=fldtype_handles,
    title="Dominant flood type",
    # loc='upper right',
    bbox_to_anchor=(1.01, 1.01),  # push slightly right & up
    fontsize=12,
    title_fontsize=16
)

ax.add_artist(legend2)

# 3. Simple dike legend (black line)
black_line = mlines.Line2D([], [], color='black', linewidth=1.2, label='Dike')
legend3 = ax.legend(handles=[black_line],
                    loc='lower left',
                    fontsize=14)
ax.add_artist(legend3)


# Save
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_all_period_maps/final_adjusted_maps"
file_name = f"gemeinde_exposed_above050_and_plain_dike_flooding_type_Jan25_100yr.png"
full_path = os.path.join(save_path, file_name)
plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
# plt.close()

### Identify exposed but underprotected Gemeinden based on their dominant flooding type.

In [ ]:
exposed_above_050_with_land_utm = exposed_above_050_with_land_utm.merge(Gemeinde_2020_100_Jul16_above_050_df_all.drop(columns='risk_level'), on='GEM_ID', how='left')

In [ ]:
exposed_above_050_with_land_utm['dike_len_km'] = exposed_above_050_with_land_utm.dike_length / 1000

In [ ]:
# The column MAX_area_Jul16_by_level_x is redundant for this part of the plot. This column was not calculated for flood risk levels above 0.5 m or above 1.5 m.
exposed_above_050_with_land_utm = exposed_above_050_with_land_utm.rename(columns={"MAX_area_Jul16_by_level_y": "MAX_area_Jul16_by_level"})

In [ ]:
exposed_above_050_with_land_utm.columns

In [ ]:
# Define flood types
flood_types = ["MAX", "CU", "FU", "PD"]

# Loop through each flood type and calculate growth
for flood_type in flood_types:
    # Build column names
    col_current = f"{flood_type}_area_by_level"
    col_base = f"{flood_type}_area_Jul16_by_level"
    col_growth = f"{flood_type.lower()}_pct_growth"  # Lowercase for the new column

    # Calculate % growth and store in a new column
    exposed_above_050_with_land_utm[col_growth] = (
        (exposed_above_050_with_land_utm[col_current] - exposed_above_050_with_land_utm[col_base])
        / (exposed_above_050_with_land_utm[col_base] + 1e-6)
    )

In [ ]:
# Define flood types
flood_types = ["MAX", "CU", "FU", "PD"]

# Loop through each flood type and calculate growth
for flood_type in flood_types:
    # Build column names
    col_current = f"{flood_type}_area_by_level"
    col_base = f"{flood_type}_area_Jul16_by_level"
    col_increase = f"{flood_type.lower()}_growth_sqkm"  # Lowercase for the new column

    exposed_above_050_with_land_utm[col_increase] = (
        exposed_above_050_with_land_utm[col_current] - exposed_above_050_with_land_utm[col_base]
    )

In [ ]:
# Assume 'OBJID' is the common identifier
dune_ids = set(intersected_polydunes_gdf["OBJID"])

# Create a new column 'has_dune': 'Yes' if OBJID is in dune_ids, otherwise 'No'
exposed_above_050_with_land_utm["has_dune"] = exposed_above_050_with_land_utm["OBJID"].apply(lambda x: True if x in dune_ids else False)

In [ ]:
exposed_above_050_with_land_utm[
    (exposed_above_050_with_land_utm['dominant_type'].isin(['fu_dominant', 'cu_dominant'])) &
    (exposed_above_050_with_land_utm['dike_len_km'] == 0)
].has_dune.value_counts()

In [ ]:
# Select Gemeinden that are fluvial or coastal dominant, and have no dike protection
no_dike = exposed_above_050_with_land_utm[
    (exposed_above_050_with_land_utm['dominant_type'].isin(['fu_dominant', 'cu_dominant'])) &
    (exposed_above_050_with_land_utm['dike_len_km'] == 0)
]

# Show how many unique Gemeinden
num_gemeinden = no_dike['OBJID'].nunique()

print(f"Number of Gemeinden exposed to fluvial or coastal flooding without any dike protection: {num_gemeinden}")

# Optional: Show the list of Gemeinde names
# print(no_dike['Gemeinde'].unique())


Exposure area from the dominant flooding type only

In [ ]:
# Create a copy of the dataframe to store flags
underprotect_df = exposed_above_050_with_land_utm[
    exposed_above_050_with_land_utm.dominant_type.isin(["fu_dominant", "cu_dominant"])
].copy().reset_index(drop=True)

# Initialize new columns
underprotect_df["under_trend"] = False
underprotect_df["pred_dike_len"] = np.nan
underprotect_df["obs_pred_diff"] = np.nan

# Map dominant_type to its respective area variable
x_var_map = {
    "cu_dominant": "CU_area_by_level",
    "fu_dominant": "FU_area_by_level"
}

# Loop over flood types
for flood_type in ["fu_dominant", "cu_dominant"]:
    # Get the appropriate x-variable for this flood type
    x_var = x_var_map[flood_type]

    # Subset the dataframe for the flood type
    subset = underprotect_df[underprotect_df["dominant_type"] == flood_type]

    # Fit OLS regression
    X = sm.add_constant(subset[x_var])
    y = subset["dike_len_km"]
    model = sm.OLS(y, X).fit()
    # print(model.summary())

    # Calculate predictions
    pred_dike_len = model.predict(X)

    # Calculate difference and under-trend flag
    diff = y - pred_dike_len
    under_trend = y < pred_dike_len

    # Store results
    underprotect_df.loc[subset.index, "pred_dike_len"] = pred_dike_len
    underprotect_df.loc[subset.index, "under_trend"] = under_trend
    underprotect_df.loc[subset.index, "obs_pred_diff"] = diff

In [ ]:
# Invert quantiles: 0 = most under-protected (largest positive diff), 9 = most over-protected (largest negative diff)
underprotect_df["obs_pred_diff_quantile"] = pd.qcut(
    underprotect_df["obs_pred_diff"],  # invert the sign
    q=10,
    labels=False,
    duplicates="drop"
)

# Get quantile thresholds
quantile_thresholds = underprotect_df["obs_pred_diff"].quantile([i/10 for i in range(1, 11)])
quantile_thresholds.index = range(1, 11)  # So threshold[1] is the 10% cutoff, etc.



In [ ]:
underprotect_df[['obs_pred_diff', 'obs_pred_diff_quantile']].head()

In [ ]:
# Set your aesthetics
sns.set(style="whitegrid")

# Mapping for titles and x-axis labels
flood_titles = {
    "fu_dominant": ("Fluvial flood", "Fluvial exposure area (>0.5 m, $km^2$)"),
    "cu_dominant": ("Coastal flood", "Coastal exposure area (>0.5 m, $km^2$)")
}

# X-variable map
x_var_map = {
    "fu_dominant": "FU_area_by_level",
    "cu_dominant": "CU_area_by_level"
}

# Define consistent colors
colors = {
    "cu_dominant": to_hex(cm.Blues(0.6)),
    "fu_dominant": to_hex(cm.Greens(0.6))
}

fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True, sharey=True)

handles_for_legend = []  # collect legend handles just once

for ax, flood_type in zip(axes, ["fu_dominant", "cu_dominant"]):
    subset = underprotect_df[underprotect_df["dominant_type"] == flood_type]
    x_var = x_var_map[flood_type]
    x = subset[x_var]
    y = subset["dike_len_km"]

    # Fit regression
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    x_vals = np.linspace(0, 10, 200)
    X_pred = sm.add_constant(x_vals)
    y_pred = model.predict(X_pred)

    # Standard errors
    se_intercept, se_slope = model.bse
    y_upper = (model.params[0] + se_intercept) + (model.params[1] + se_slope) * x_vals
    y_lower = (model.params[0] - se_intercept) + (model.params[1] - se_slope) * x_vals

    # Scatter: base points
    sc = ax.scatter(x, y, s=20, alpha=0.6, color=colors[flood_type])

    # Highlight most under-protected (quantile = 0)
    worst = subset[subset["obs_pred_diff_quantile"] == 0]
    circ = ax.scatter(
        worst[x_var],
        worst["dike_len_km"],
        s=70, facecolors="none", edgecolors="red", linewidth=1.8,
        label="Bottom 10 percentile (most under-protected)"
    )

    # Regression lines
    reg_line, = ax.plot(x_vals, y_pred, color=colors[flood_type])
    upper_line, = ax.plot(x_vals, y_upper, color="black", linestyle="--")
    lower_line, = ax.plot(x_vals, y_lower, color="black", linestyle="--", label=r"$(\beta \pm 1\,\mathrm{SE})$")

    # Titles & limits
    title, xlabel = flood_titles[flood_type]
    ax.set_xlabel(xlabel)
    ax.set_title(f"{title}")
    ax.set_xlim(-0.5, 10.5)
    ax.set_ylim(-5, 105)

    # Equation annotation with coefficients ±1SE
    intercept, slope = model.params
    eq_text = (
        r"$\hat{y} = "
        f"{intercept:.2f} \,\pm\, {se_intercept:.2f} \;+\; "
        f"({slope:.2f} \,\pm\, {se_slope:.2f}) \, x$"
    )
    ax.text(0.05, 0.95, eq_text,
            transform=ax.transAxes,
            fontsize=10, va="top", ha="left",
            bbox=dict(facecolor="white", alpha=0.6, edgecolor="none"))

    # Collect handles just once (from the last loop is enough)
    handles_for_legend = [lower_line, circ]

# Shared y-axis label
axes[0].set_ylabel("Dike length (km)")

# Put legend centered below both plots
fig.legend(handles=handles_for_legend,
           loc="lower center", ncol=4, frameon=False)

plt.tight_layout(rect=[0, 0.08, 1, 1])  # leave space at bottom for legend
plt.show()


In [ ]:
# Map: for each row, get the threshold of the *next* quantile
def get_next_quantile_threshold(q):
    if q < 9:
        return quantile_thresholds[q + 1]
    else:
        return np.nan  # No next quantile if already in top group

underprotect_df["next_quantile_threshold"] = underprotect_df["obs_pred_diff_quantile"].astype(int).apply(get_next_quantile_threshold)

# Compute the log-km needed to reach the next quantile threshold
underprotect_df["km_to_reach_nextq"] = (
    underprotect_df["next_quantile_threshold"] - underprotect_df["obs_pred_diff"]
).where(underprotect_df["next_quantile_threshold"].notna(), 0.0)


In [ ]:
# Assume 'OBJID' is the common identifier
dune_ids = set(intersected_polydunes_gdf["OBJID"])

# Create a new column 'has_dune': 'Yes' if OBJID is in dune_ids, otherwise 'No'
underprotect_df["has_dune"] = underprotect_df["OBJID"].apply(lambda x: True if x in dune_ids else False)

In [ ]:
underprotect_df[['MAX_area_by_level', 'CU_area_by_level', 'FU_area_by_level', 'PD_area_by_level',
                 'under_trend', 'obs_pred_diff', 'obs_pred_diff_quantile', 'next_quantile_threshold', 'km_to_reach_nextq']].head()

In [ ]:
underprotect_df[underprotect_df.under_trend==True].dominant_type.value_counts()

In [ ]:
underprotect_df[underprotect_df.under_trend == True].groupby(['dominant_type', 'has_dune']).size().reset_index(name='count')

In [ ]:
underprotect_df[underprotect_df.obs_pred_diff_quantile == 0].has_dune.value_counts()

In [ ]:
underprotect_df[(underprotect_df.obs_pred_diff_quantile == 0)].km_to_reach_nextq.sum().round(2)

In [ ]:
# Filter and project the data
gemeinde_plot = underprotect_df[(underprotect_df.under_trend == True) & (underprotect_df.has_dune == False)].drop(columns='intersecting_dikes').copy().to_crs(epsg=3857)
gemeinde_boundary = Gemeinde_2020_100_jan25_above_050_all_utm.drop(columns='intersecting_dikes').copy().to_crs(epsg=3857)

gemeinde_plot2 = gemeinde_plot[gemeinde_plot['max_pct_growth'].notna()].copy()

# Quantile classification (e.g., 5 quantiles)
qntl = mapclassify.Quantiles(gemeinde_plot2['max_pct_growth'], k=5)

gemeinde_plot2['growth_bin'] = np.nan
gemeinde_plot2.loc[gemeinde_plot2['max_pct_growth'].notna(), 'growth_bin'] = qntl.yb
gemeinde_plot2['growth_bin'] = gemeinde_plot2['growth_bin'].astype('Int64')

# Red colormap
cmap_steps = 5
colormap = cm.Reds(np.linspace(0, 1.0, cmap_steps))

# Assign colors based on bins
def get_growth_color(row):
    bin_idx = row['growth_bin']
    if pd.isna(bin_idx):
        return "#d3d3d3"  # Light grey for NaNs
    try:
        bin_idx = int(bin_idx)
        return to_hex(colormap[bin_idx])
    except Exception as e:
        print(f"Error on row {row.name}: {e}")
        return "#000000"

gemeinde_plot2['color'] = gemeinde_plot2.apply(get_growth_color, axis=1)

# Plotting
fig, ax = plt.subplots(figsize=(14, 12))
gemeinde_plot2.plot(ax=ax, color=gemeinde_plot2['color'], edgecolor=None, linewidth=0.5)

# Gemeinde boundary
gemeinde_boundary.boundary.plot(ax=ax, color='grey', linewidth=0.05)

# Basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gemeinde_plot.crs)

# Title and cleanup
# ax.set_title("Under-Protected Gemeinden — Max % Growth in Exposure", fontsize=14)
ax.axis('off')

# North arrow and scale bar
add_north_arrow(ax, scale=0.7, xlim_pos=0.925, ylim_pos=0.90, color='black', text_scaler=4, text_yT=-1.25)
scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
ax.add_artist(scale1)

circles = []
# Iterate through each bin and set the label
for i, (lower, upper) in enumerate(zip([gemeinde_plot2['max_pct_growth'].min()] + list(qntl.bins[:-1]), qntl.bins)):
    if i == len(qntl.bins) - 1:
        label = f"{lower:.1%} ~ 100%+"
    else:
        label = f"{lower:.1%} ~ {upper:.1%}"

    circle_marker = Line2D(
        [0], [0],
        marker='o',
        color='None',
        markerfacecolor=to_hex(colormap[i]),
        markeredgecolor='None',
        markersize=10,
        label=label
    )
    circles.append(circle_marker)

legend1 = ax.legend(handles=circles, title="Exposure \ngrowth", loc='upper left', fontsize=12, title_fontsize=16)

ax.add_artist(legend1)

# Save the figure
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_all_period_maps/final_adjusted_maps"
file_name = "gemeinde_exposed_above050_max_pct_growth_underprotected_no_dune_Jul16_100yr_5quantile.png"
full_path = os.path.join(save_path, file_name)
plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
# plt.close()


In [ ]:
underprotect_df['LAN_NAME_MOD'] = underprotect_df['LAN_NAME'].replace(
                                      {'Bremen': 'City States', 'Hamburg': 'City States', 'Berlin': 'City States'}
                                  )

In [ ]:
protected_exposed_above_050_cu_fu_utm = exposed_above_050_with_land_utm[(exposed_above_050_with_land_utm.dike_length > 0) & (exposed_above_050_with_land_utm.dominant_type.isin(['cu_dominant', 'fu_dominant']))]
protected_exposed_above_050_cu_fu_utm['LAN_NAME_MOD'] = protected_exposed_above_050_cu_fu_utm['LAN_NAME'].replace(
    {'Bremen': 'City States', 'Hamburg': 'City States', 'Berlin': 'City States'}
)

In [ ]:
# Calculate exposure per km of dike
protected_exposed_above_050_cu_fu_utm["exposure_per_km_dike"] = protected_exposed_above_050_cu_fu_utm["MAX_area_by_level"] / protected_exposed_above_050_cu_fu_utm["dike_len_km"]

# Group by state and dominant flooding type, then calculate the mean
state_floodtype_means = protected_exposed_above_050_cu_fu_utm.groupby(["LAN_NAME_MOD", "dominant_type"])["exposure_per_km_dike"].mean().reset_index()

# Rename the resulting column for clarity
state_floodtype_means.rename(columns={"exposure_per_km_dike": "avg_exposure_per_km_dike"}, inplace=True)

# Display the result
print(state_floodtype_means)


In [ ]:
improve_underprotect_df = underprotect_df.merge(state_floodtype_means, on=['LAN_NAME_MOD', 'dominant_type'], how='left')

In [ ]:
improve_underprotect_df['avg_exposure_reduction'] = improve_underprotect_df.km_to_reach_nextq * improve_underprotect_df.avg_exposure_per_km_dike

In [ ]:
improve_underprotect_df[improve_underprotect_df.obs_pred_diff_quantile==0].avg_exposure_reduction.sum().round(2)

### Plot protection versus exposure on a log10 scale to improve visualization of their correlation.

In [ ]:
protected_100_jan25_above_050 = Gemeinde_2020_100_jan25_above_050_all_utm[Gemeinde_2020_100_jan25_above_050_all_utm.dike_length>0].reset_index(drop=True)

In [ ]:
protected_100_jan25_above_050['lg_dike_length_km'] = np.log10(protected_100_jan25_above_050.dike_length/1000)

In [ ]:
protected_exposure_100_jan25_above_050 = protected_100_jan25_above_050[protected_100_jan25_above_050.MAX_area_by_level>0]

In [ ]:
protected_exposure_100_jan25_above_050['lg_MAX_area_by_level'] = np.log10(protected_exposure_100_jan25_above_050.MAX_area_by_level)
protected_exposure_100_jan25_above_050['lg_CU_area_by_level'] = np.log10(protected_exposure_100_jan25_above_050.CU_area_by_level)
protected_exposure_100_jan25_above_050['lg_FU_area_by_level'] = np.log10(protected_exposure_100_jan25_above_050.FU_area_by_level)
protected_exposure_100_jan25_above_050['lg_PD_area_by_level'] = np.log10(protected_exposure_100_jan25_above_050.PD_area_by_level)

In [ ]:
protected_exposure_100_jan25_above_050['dike_length_km'] = protected_exposure_100_jan25_above_050.dike_length/1000

In [ ]:
# Set your aesthetics
sns.set(style="whitegrid")

# Map flood type to its corresponding x-variable
x_var_map = {
    "cu_dominant": "lg_CU_area_by_level",
    "fu_dominant": "lg_FU_area_by_level",
    "pd_dominant": "lg_PD_area_by_level"
}

# Set the flood label mapping
flood_label_map = {
    "cu_dominant": "Coastal Flood",
    "fu_dominant": "Fluvial Flood",
    "pd_dominant": "Pluvial Flood"
}
palette = sns.color_palette("Set2")
regression_color_map = {
    "pd_dominant": to_hex(cm.Purples(0.6)),  # Pluvial – purple
    "fu_dominant": to_hex(cm.Greens(0.6)),   # Fluvial – green
    "cu_dominant": to_hex(cm.Blues(0.6))     # Coastal – blue
}

# Calculate slopes
slope_dict = {}
for flood_type in protected_exposure_100_jan25_above_050["dominant_type"].unique():
    x_var = x_var_map.get(flood_type, "lg_MAX_area_by_level")
    subset = protected_exposure_100_jan25_above_050[protected_exposure_100_jan25_above_050["dominant_type"] == flood_type]
    slope, intercept, r, pval, stderr = scipy.stats.linregress(subset[x_var], subset["lg_dike_length_km"])
    slope_dict[flood_type] = abs(slope)  # use abs(slope) if you want magnitude

# Sort flood types
sorted_flood_types = sorted(slope_dict.keys(), key=lambda x: slope_dict[x])

protected_exposure_100_jan25_above_050["dominant_type_ordered"] = pd.Categorical(
    protected_exposure_100_jan25_above_050["dominant_type"],
    categories=sorted_flood_types,
    ordered=True
)

# Create the FacetGrid
g = sns.FacetGrid(
    protected_exposure_100_jan25_above_050,
    col="dominant_type_ordered",
    hue="dominant_type",
    height=5,
    aspect=1
)

# Loop through and draw scatter + regression per facet
for ax, flood_type in zip(g.axes.flat, g.col_names):
    subset = protected_exposure_100_jan25_above_050[
        protected_exposure_100_jan25_above_050["dominant_type"] == flood_type
    ]
    x_var = x_var_map.get(flood_type, "lg_MAX_area_by_level")
    color = regression_color_map.get(flood_type, 'black')

    # Scatter + regression
    sns.regplot(
        data=subset,
        x=x_var,
        y="lg_dike_length_km",
        ax=ax,
        scatter_kws={'s': 20, 'color': color},
        line_kws={'color': color, 'lw': 2}
    )

    # Regression statistics
    slope, intercept, r, pval, stderr = scipy.stats.linregress(
        subset[x_var], subset["lg_dike_length_km"]
    )

    eq_text = f"y = {intercept:.2f} + {slope:.2f}x\nR² = {r**2:.2f}"
    ax.text(
        0.05, 0.95, eq_text,
        transform=ax.transAxes, ha='left', va='top',
        fontsize=10, bbox=dict(facecolor='white', alpha=0.6, edgecolor='none')
    )

    # Sentence-case labels
    flood_label = flood_label_map.get(flood_type, flood_type).lower()
    ax.set_title(flood_label.capitalize())
    ax.set_xlabel(f"Log {flood_label} exposure (>0.5 m)", fontsize=11)
    ax.set_ylabel("Log dike length (km)", fontsize=11)

    # Format ticks as powers of ten with superscripts
    sci_formatter = ticker.FuncFormatter(lambda x, _: f"$10^{{{int(x)}}}$")
    ax.xaxis.set_major_formatter(sci_formatter)
    ax.yaxis.set_major_formatter(sci_formatter)

# Save plot
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_protection_plots/final_adjustments"
file_name = f"gemeinde_exposed_by_flood_type_above050_and_dike_length_Jan25_100yr.png"
full_path = os.path.join(save_path, file_name)
plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')

plt.show()
